# Demographic Data Visulizations

#### Setup

In [ ]:
import os
import arcpy
from arcgis.gis import GIS
from arcgis.mapping import WebMap
from arcgis.features import FeatureSet, GeoAccessor, GeoSeriesAccessor, FeatureLayer, FeatureLayerCollection
# import datetime as dt
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import plotly.offline as py
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px
import plotly.figure_factory as ff
from plotly.offline import iplot, init_notebook_mode
from plotly.subplots import make_subplots
#This needs to be manually installed to your environment using: conda install -c conda-forge python-kaleido=0.1.0
import kaleido
import plotly.io as pio
py.init_notebook_mode()
workspace = os.path.abspath('')

In [ ]:
# Connect to TRPA Enterprise GIS Portal
portal_url = "https://maps.trpa.org/portal"
gis = GIS(portal_url)

# plotly settings
init_notebook_mode(connected=False)

# pandas settings
pd.options.plotting.backend = "plotly"
# set data frame display options
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.options.display.float_format = '{:,.2f}'.format

# set overwrite to true
arcpy.env.overwriteOutput = True

# in memory output file path
wk_memory = "memory/"

# set style variables
template = 'plotly_white'
font     = 'Calibri'
# qualitative diverging color scheme
colors = ["#a6cee3",
            "#1f78b4",
            "#b2df_non_tahoe8a",
            '#33a02c',
            '#fb9a99',
            '#e31a1c',
            '#fdbf6f',
            '#ff7f00',
            '#cab2d6',
            '#6a3d9a']
# colors for tables
headerColor = '#6680a8'
rowColor = 'white'
lastrowColor = '#eeeeee'

In [ ]:
def get_census_data(featureset):
    service_id = {
        'raw_data':'28',
        'summaries':'18'
    }

    service_number = service_id.get(featureset) 
    service_url = 'https://maps.trpa.org/server/rest/services/Demographics/MapServer/'+service_number

    feature_layer = FeatureLayer(service_url)
    query_result = feature_layer.query()
    # Convert the query result to a list of dictionaries
    feature_list = query_result.features

    # Create a pandas DataFrame from the list of dictionaries
    all_data = pd.DataFrame([feature.attributes for feature in feature_list])

    return all_data

In [ ]:
census_data = get_census_data('raw_data')
census_data = census_data.loc[census_data['county']!='510']
summary_data = get_census_data('summaries')
county_zone = {
    '061':'North Lake',
    '031':'North Lake',
    '005': 'South Lake',
    '017':'South Lake'
}

census_data['Zone'] = census_data['county'].map(county_zone)

## Census Charts

### Population - Decennial 1990-2020

In [ ]:
dfTotalPopulation = summary_data.loc[(summary_data['Year'].isin([1990, 2000, 2010, 2020]))&
  (summary_data['Geography']=='Basin')&
  (summary_data['Category']=='Total Population')&
  (summary_data['Source']!='acs/acs5')]

dfTotalPopulation = dfTotalPopulation.sort_values(by=['Year'], ignore_index=True)

fig_total_pop = px.line(dfTotalPopulation, 
                        x="Year", y="Value",
                        labels=dict(Year='Year', Value='Population'),
                        title="Tahoe Decennial population 1990-2020")

fig_total_pop.update_traces(mode='markers+lines', 
                    
                     hovertemplate='Census Year: %{x} <br> Population: %{y:,.0f}'
                    )
fig_total_pop.update_layout(font_family=font,
                     template=template,
                     hoverlabel=dict(
                     bgcolor='white',
                       font=dict(color='#6680A8') 
                     ),
                     title=dict(
                                x=0.5,
                                xanchor='center',
                                font=dict(size=32)),
                     xaxis=dict(
                                tick0=1990, 
                                dtick=10,
                                title = 'Year',
							                  title_font_size = 24,
                                tickfont = dict(size=20)),
                     yaxis=dict(
                                range=[0,65000],
                                title = 'Population',
							                  title_font_size = 24,
                                tickfont = dict(size=20)))

fig_total_pop.show()


In [ ]:
fig_total_pop.write_html(os.path.join(workspace, "Open_Data_Figures/Demographics_Tahoe_Population_Line.html"))

## Population of surrounding counties

In [ ]:
counties = ['El Dorado County','Douglas County', 'Placer County', 'Washoe County', 'Carson City County', 'Basin']
dfTotalPopulation = summary_data.loc[(summary_data['Year'].isin([1990, 2000, 2010, 2020]))&
  (summary_data['Geography'].isin(counties))&
  (summary_data['Category']=='Total Population')&
  (summary_data['Source']!='acs/acs5')]
color_mapping = {
    "Tahoe Basin": '#00407a',
    'Surrounding 5-Counties': '#996308'
}
county_map = {
    'El Dorado County': "Surrounding 5-Counties",
    'Douglas County': "Surrounding 5-Counties", 
    'Placer County': "Surrounding 5-Counties", 
    'Washoe County': "Surrounding 5-Counties", 
    'Carson City County': "Surrounding 5-Counties", 
    'Basin':'Tahoe Basin'
}

dfTotalPopulation['grouped_geography'] = dfTotalPopulation['Geography'].map(county_map)

dfTotalPopulation= dfTotalPopulation.groupby(['Year', 'grouped_geography'], as_index = False)['Value'].sum()
dfTotalPopulation = dfTotalPopulation.sort_values(by=['Year', 'grouped_geography'], ascending=[True, False], ignore_index=True)

fig_bar = px.bar(dfTotalPopulation, x="Year", y="Value", color='grouped_geography',
                labels=dict(Year='Census Year', Value='Population'), color_discrete_map= color_mapping,
                title="Decennial Population 1990-2020", barmode='group'
)
fig_bar.update_layout(font_family=font,
                 template=template,
                 bargroupgap = 0.20,
                    hoverlabel=dict(
                    bgcolor='white',
                       font=dict(color='#6680A8') 
                    ),
    legend=dict(
        title='',  # Legend title
        font=dict(size=24)
    ),
                     title=dict(
                                x=0.5,
                                xanchor='center',
                                font=dict(size=32)),
                     xaxis=dict(
                                tick0=1990, 
                                dtick=10,
                                title = 'Census Year',
							                  title_font_size = 24,
                                tickfont = dict(size=20)),
                     yaxis=dict(
                                title = 'Population',
							                  title_font_size = 24,
                                tickfont = dict(size=20)))
fig_bar.update_traces(hovertemplate='Census Year: %{x} <br> Population: %{y:,.0f} <extra></extra>')
fig_bar.update_traces(width=3.5)
fig_bar.show()



fig_bar.write_html(os.path.join(workspace, "Open_Data_Figures\Demographics_NonTahoe_Population_Bar.html"))

In [ ]:
dfTotalPopulation = summary_data.loc[(summary_data['Year'].isin([1990, 2000, 2010, 2020]))&
  (summary_data['Geography']=='Basin')&
  (summary_data['Category']=='Total Population')&
  (summary_data['Source']!='acs/acs5')]

dfTotalPopulation = dfTotalPopulation.sort_values(by=['Year'], ignore_index=True)

fig_bar = px.bar(dfTotalPopulation, x="Year", y="Value",
                labels=dict(Year='Census Year', Value='Population'),
                title="Tahoe Decennial Population 1990-2020",
)
fig_bar.update_layout(font_family=font,
                 template=template,
                    hoverlabel=dict(
                    bgcolor='white',
                       font=dict(color='#6680A8') 
                    ),
                     title=dict(
                                x=0.5,
                                xanchor='center',
                                font=dict(size=32)),
                     xaxis=dict(
                                tick0=1990, 
                                dtick=10,
                                title = 'Census Year',
							                  title_font_size = 24,
                                tickfont = dict(size=20)),
                     yaxis=dict(
                                range=[0,65000],
                                title = 'Population',
							                  title_font_size = 24,
                                tickfont = dict(size=20)))
fig_bar.update_traces(marker_color='#2F3F56',
                     hovertemplate='Census Year: %{x} <br> Population: %{y:,.0f} ')
fig_bar.update_traces(width=4)
fig_bar.show()



fig_bar.write_html(os.path.join(workspace, "Open_Data_Figures\Demographics_Tahoe_Population_Bar.html"))

### Bring in Census data for age 

In [ ]:
dfPopulation = census_data.loc[(census_data['variable_name'].isin(['Population Total','Total Population']))&(census_data['sample_level']=='tract')&(census_data['dataset']!='acs/acs5')]
dfPopulation['Zone'] = None
dfPopulation.loc[dfPopulation['county'].isin(['061','031']), 'Zone'] = "North Lake"
dfPopulation.loc[dfPopulation['county'].isin(['005','017']), 'Zone'] = "South Lake"

df = dfPopulation.groupby(['variable_code', 'year_sample', 'Zone'], as_index=False).sum(['value'])
fig = px.bar(df, x="year_sample", y="value", color="Zone", title="Total Population")
fig.show()

### Age Distribution

#### Tahoe Age Pyramid

In [ ]:
# get data
dfCensus = census_data
dfAgeSex = dfCensus.loc[dfCensus['variable_category'].isin(['TRPA Census Age Sex Categories Grouped: Age'])]
dfAgeSex_Tahoe = dfAgeSex.loc[dfAgeSex['sample_level']=='tract']
# group by variable name/code/year and sum values
df = dfAgeSex_Tahoe.groupby(['variable_name','variable_code','year_sample'], as_index= False).sum(['value'])

# ordered list to sort age category by
sort_list = [ 'Under 5', '5 to 9','10 to 14', '15 to 19', '20 to 24', 
            '25 to 29', '30 to 34', '35 to 39', '40 to 44', '45 to 49', 
            '50 to 54', '55 to 59', '60 to 64', '65 to 69', '70 to 74', 
            '75 to 79', '80 to 84', 'Over 85']

# setup new fields
# gender is tagged on the end of variable name. split it out after the colon
df['Gender'] = df['variable_name'].str.split(':').str[-1]
# age is the first part of the variable name string
df['Age']    = df['variable_name'].str.split(':').str[0]
# drop the rows that represent the Totals
df = df[df.Age != 'Total']
# setup a new Year column and format it
df['Year'] = pd.to_datetime(df['year_sample'], format='%Y') 
df['Year']   = df['Year'].dt.strftime('%Y')
# setup a new Count field based on the summed values
df['Count'] = df['value']

# cast count as integer type
df = df.astype({'Count':int})

# keep the new columsn
df = df[['Age','Year','Gender','Count']]

# set Age to be a categorical field with the same order as the list
df['Age'] = pd.Categorical(df['Age'], categories = sort_list)
# sort the age category
df = df.sort_values(by='Age')

pivotTable = pd.pivot_table(df, values='Count', index=['Age','Year'],
                       columns=['Gender'], aggfunc=np.sum)
flat = pd.DataFrame(pivotTable.to_records())
df = flat
# remove space in column names 'Male' and "Female"
df.columns = df.columns.str.replace(' ', '')


# Creating instance of the figure
fig = make_subplots(rows=1, cols=4, shared_yaxes=True)

 # Filter by single year
df1990 = df[df['Year']== '1990']
df2000 = df[df['Year']== '2000']
df2010 = df[df['Year']== '2010']
df2020 = df[df['Year']== '2020']

# set x and y variables for plot
y_age90    = df1990['Age']
x_male90   = df1990['Male']
x_female90 = df1990['Female'] * -1

# set x and y variables for plot
y_age00    = df2000['Age']
x_male00   = df2000['Male']
x_female00 = df2000['Female'] * -1

# set x and y variables for plot
y_age10    = df2010['Age']
x_male10   = df2010['Male']
x_female10 = df2010['Female'] * -1

# set x and y variables for plot
y_age20    = df2020['Age']
x_male20   = df2020['Male']
x_female20 = df2020['Female'] * -1

mcolor = '#405d74'
fcolor = '#679ab8'
# Adding Male data to the figure
fig.add_trace(go.Bar(y= y_age90, 
					x = x_male90,
					name = 'Male',
					orientation = 'h',
					marker_color=mcolor,
					legendgroup = 'Male',
					customdata = np.stack((df1990['Male'], df1990['Age']), axis=-1),
					hovertemplate='%{customdata[0]:,.0f} males<extra></extra>'
					),
					row=1, col=1)

# Adding Female data to the figure
fig.add_trace(go.Bar(y = y_age90, 
					x = x_female90,
					name = 'Female', 
					orientation = 'h', 
					marker_color=fcolor,
					legendgroup ='Female',
					customdata = np.stack((df1990['Female'], df1990['Age']), axis=-1),
					hovertemplate='%{customdata[0]:,.0f} females<extra></extra>'
					),
					row=1, col=1)


# Adding Male data to the figure
fig.add_trace(go.Bar(y= y_age00, 
					x = x_male00,
					name = 'Male',
					orientation = 'h',
					marker_color=mcolor,
					legendgroup = 'Male',
                    showlegend  = False,
					customdata = np.stack((df2000['Male'], df2000['Age']), axis=-1),
					hovertemplate='%{customdata[0]:,.0f} males<extra></extra>'
					),
					row=1, col=2)

# Adding Female data to the figure
fig.add_trace(go.Bar(y = y_age00, 
					x = x_female00,
					name = 'Female', 
					orientation = 'h', 
					marker_color=fcolor,
					legendgroup ='Female',
                    showlegend  = False,
					customdata = np.stack((df2000['Female'], df2000['Age']), axis=-1),
					hovertemplate='%{customdata[0]:,.0f} females<extra></extra>'
					),
					row=1, col=2)

# Adding Male data to the figure
fig.add_trace(go.Bar(y= y_age10, 
					x = x_male10,
					name = 'Male',
					orientation = 'h',
					marker_color=mcolor,
					legendgroup = 'Male',
					showlegend  = False,
					customdata = np.stack((df2010['Male'], df2010['Age']), axis=-1),
					hovertemplate='%{customdata[0]:,.0f} males<extra></extra>'
					),
					row=1, col=3)

# Adding Female data to the figure
fig.add_trace(go.Bar(y = y_age10, 
					x = x_female10,
					name = 'Female', 
					orientation = 'h', 
					marker_color=fcolor,
					legendgroup = 'Female',
					showlegend  = False,
					customdata = np.stack((df2010['Female'], df2010['Age']), axis=-1),
					hovertemplate='%{customdata[0]:,.0f} females<extra></extra>'
					),
					row=1, col=3)

# Adding Male data to the figure
fig.add_trace(go.Bar(y= y_age20, 
					x = x_male20,
					name = 'Male',
					orientation = 'h',
					marker_color=mcolor,
					legendgroup = 'Male',
					showlegend  = False,
					customdata = np.stack((df2020['Male'], df2020['Age']), axis=-1),
					hovertemplate='%{customdata[0]:,.0f} males<extra></extra>'
					),
					row=1, col=4)

# Adding Female data to the figure
fig.add_trace(go.Bar(y = y_age20, 
					x = x_female20,
					name = 'Female', 
					orientation = 'h', 
					marker_color=fcolor,
					legendgroup ="Female",
					showlegend  = False,
					customdata = np.stack((df2020['Female'], df2020['Age']), axis=-1),
					hovertemplate='%{customdata[0]:,.0f} females<extra></extra>'
					),
					row=1, col=4)


# Updating the layout
fig.update_layout(title = 'Age Profile: 5 Year Increments',
				title_x=0.5,
				title_font_size = 22, barmode = 'relative',
				bargap = 0.0, bargroupgap = 0,
				hovermode="y unified",
				xaxis = dict(tickvals = [-6000, -4000, -2000,
										0, 2000, 4000, 6000],
								
							ticktext = ['6k', '4k', '2k', '0',
										'2k', '4k', '6k'],
							title_font_size = 14),
				xaxis2 = dict(tickvals = [-6000, -4000, -2000,
										0, 2000, 4000, 6000],
							ticktext = ['6k', '4k', '2k', '0',
										'2k', '4k', '6k'],
							title_font_size = 14),
				xaxis3 = dict(tickvals = [-6000, -4000, -2000,
										0, 2000, 4000, 6000],
							ticktext = ['6k', '4k', '2k', '0',
										'2k', '4k', '6k'],
							title_font_size = 14),
                xaxis4 = dict(tickvals = [-6000, -4000, -2000,
										0, 2000, 4000, 6000],
							ticktext = ['6k', '4k', '2k', '0',
										'2k', '4k', '6k'],
							title_font_size = 14),
				yaxis = dict(					
							title = 'Age',
							title_font_size = 14)			
				)


# edit axis labels
fig['layout']['xaxis']['title'] ='1990 Decennial Census'
fig['layout']['xaxis2']['title']='2000 Decennial Census'
fig['layout']['xaxis3']['title']='2010 Decennial Census'
fig['layout']['xaxis4']['title']='2020 Decennial Census'

fig.show()
fig.write_html(os.path.join(workspace, "Demographics_Tahoe_AgeProfile_DecadeSubplots.html"))

#### Non-Tahoe Age Pyramids

In [ ]:
# get data
dfCensus = census_data
dfAgeSex = dfCensus.loc[dfCensus['variable_category'].isin(['TRPA Census Age Sex Categories Grouped: Age'])]
dfAgeSex_Tahoe = dfAgeSex.loc[dfAgeSex['sample_level']=='County']
# group by variable name/code/year and sum values
df = dfAgeSex_Tahoe.groupby(['variable_name','variable_code','year_sample'], as_index= False).sum(['value'])

# ordered list to sort age category by
sort_list = [ 'Under 5', '5 to 9','10 to 14', '15 to 19', '20 to 24', 
            '25 to 29', '30 to 34', '35 to 39', '40 to 44', '45 to 49', 
            '50 to 54', '55 to 59', '60 to 64', '65 to 69', '70 to 74', 
            '75 to 79', '80 to 84', 'Over 85']

# setup new fields
# gender is tagged on the end of variable name. split it out after the colon
df['Gender'] = df['variable_name'].str.split(':').str[-1]
# age is the first part of the variable name string
df['Age']    = df['variable_name'].str.split(':').str[0]
# drop the rows that represent the Totals
df = df[df.Age != 'Total']
# setup a new Year column and format it
df['Year'] = pd.to_datetime(df['year_sample'], format='%Y') 
df['Year']   = df['Year'].dt.strftime('%Y')
# setup a new Count field based on the summed values
df['Count'] = df['value']

# cast count as integer type
df = df.astype({'Count':int})

# keep the new columsn
df = df[['Age','Year','Gender','Count']]

# set Age to be a categorical field with the same order as the list
df['Age'] = pd.Categorical(df['Age'], categories = sort_list)
# sort the age category
df = df.sort_values(by='Age')



pivotTable = pd.pivot_table(df, values='Count', index=['Age','Year'],
                       columns=['Gender'], aggfunc=np.sum)
flat = pd.DataFrame(pivotTable.to_records())
df = flat
# remove space in column names 'Male' and "Female"
df.columns = df.columns.str.replace(' ', '')


df['Total_Male'] = df.groupby(['Year'], as_index=False)['Male'].transform('sum')
df['Total_Female'] = df.groupby(['Year'], as_index=False)['Female'].transform('sum')

df['Male_proportion'] = df['Male']/df['Total_Male']
df['Female_proportion'] = df['Female']/df['Total_Female']

df['Male_Percentage'] =  round((df['Male_proportion']*100),2)
df['Female_Percentage'] = round((df['Female_proportion']*100),2)



# Creating instance of the figure
fig = make_subplots(rows=1, cols=4, shared_yaxes=True)

 # Filter by single year
df1990 = df[df['Year']== '1990']
df2000 = df[df['Year']== '2000']
df2010 = df[df['Year']== '2010']
df2020 = df[df['Year']== '2020']

# set x and y variables for plot
y_age90    = df1990['Age']
x_male90   = df1990['Male_proportion']
x_female90 = df1990['Female_proportion'] * -1

# set x and y variables for plot
y_age00    = df2000['Age']
x_male00   = df2000['Male_proportion']
x_female00 = df2000['Female_proportion'] * -1

# set x and y variables for plot
y_age10    = df2010['Age']
x_male10   = df2010['Male_proportion']
x_female10 = df2010['Female_proportion'] * -1

# set x and y variables for plot
y_age20    = df2020['Age']
x_male20   = df2020['Male_proportion']
x_female20 = df2020['Female_proportion'] * -1

mcolor = '#405d74'
fcolor = '#679ab8'
# Adding Male data to the figure
fig.add_trace(go.Bar(y= y_age90, 
					x = x_male90,
					name = 'Male',
					orientation = 'h',
					marker_color=mcolor,
					legendgroup = 'Male',
					customdata = np.stack((df1990['Male_proportion'], df1990['Age']), axis=-1),
					hovertemplate='%{customdata[0]:.1%} of males<extra></extra>'
					),
					row=1, col=1)

# Adding Female data to the figure
fig.add_trace(go.Bar(y = y_age90, 
					x = x_female90,
					name = 'Female', 
					orientation = 'h', 
					marker_color=fcolor,
					legendgroup ='Female',
					customdata = np.stack((df1990['Female_proportion'], df1990['Age']), axis=-1),
					hovertemplate='%{customdata[0]:.1%} of females<extra></extra>'
					),
					row=1, col=1)


# Adding Male data to the figure
fig.add_trace(go.Bar(y= y_age00, 
					x = x_male00,
					name = 'Male',
					orientation = 'h',
					marker_color=mcolor,
					legendgroup = 'Male',
                    showlegend  = False,
					customdata = np.stack((df2000['Male_proportion'], df2000['Age']), axis=-1),
					hovertemplate='%{customdata[0]:.1%} of males<extra></extra>'
					),
					row=1, col=2)

# Adding Female data to the figure
fig.add_trace(go.Bar(y = y_age00, 
					x = x_female00,
					name = 'Female', 
					orientation = 'h', 
					marker_color=fcolor,
					legendgroup ='Female',
                    showlegend  = False,
					customdata = np.stack((df2000['Female_proportion'], df2000['Age']), axis=-1),
					hovertemplate='%{customdata[0]:.1%} of females<extra></extra>'
					),
					row=1, col=2)

# Adding Male data to the figure
fig.add_trace(go.Bar(y= y_age10, 
					x = x_male10,
					name = 'Male',
					orientation = 'h',
					marker_color=mcolor,
					legendgroup = 'Male',
					showlegend  = False,
					customdata = np.stack((df2010['Male_proportion'], df2010['Age']), axis=-1),
					hovertemplate='%{customdata[0]:.1%} of males<extra></extra>'
					),
					row=1, col=3)

# Adding Female data to the figure
fig.add_trace(go.Bar(y = y_age10, 
					x = x_female10,
					name = 'Female', 
					orientation = 'h', 
					marker_color=fcolor,
					legendgroup = 'Female',
					showlegend  = False,
					customdata = np.stack((df2010['Female_proportion'], df2010['Age']), axis=-1),
					hovertemplate='%{customdata[0]:.1%} of females<extra></extra>'
					),
					row=1, col=3)

# Adding Male data to the figure
fig.add_trace(go.Bar(y= y_age20, 
					x = x_male20,
					name = 'Male',
					orientation = 'h',
					marker_color=mcolor,
					legendgroup = 'Male',
					showlegend  = False,
					customdata = np.stack((df2020['Male_proportion'], df2020['Age']), axis=-1),
					hovertemplate='%{customdata[0]:.1%} of males<extra></extra>'
					),
					row=1, col=4)

# Adding Female data to the figure
fig.add_trace(go.Bar(y = y_age20, 
					x = x_female20,
					name = 'Female', 
					orientation = 'h', 
					marker_color=fcolor,
					legendgroup ="Female",
					showlegend  = False,
					customdata = np.stack((df2020['Female_proportion'], df2020['Age']), axis=-1),
					hovertemplate='%{customdata[0]:.1%} of females<extra></extra>'
					),
					row=1, col=4)
axis_format = dict(tickvals = [-.10, -.05, 0,
										.05, .10],
							ticktext = ['10%', '5%', '0', '5%',
										'10%'],
							title_font_size = 14, range=[-.15,.15])

# Updating the layout
fig.update_layout(title = 'Age Profile: Surrounding Five Counties',
				title_x=0.5,
				title_font_size = 22, barmode = 'relative',
				bargap = 0.0, bargroupgap = 0,
				hovermode="y unified",
				xaxis = axis_format,
				xaxis2 = axis_format,
				xaxis3 = axis_format,
                xaxis4 = axis_format,
				yaxis = dict(					
							title = 'Age',
							title_font_size = 14)			
				)


# edit axis labels
fig['layout']['xaxis']['title'] ='1990'
fig['layout']['xaxis2']['title']='2000'
fig['layout']['xaxis3']['title']='2010'
fig['layout']['xaxis4']['title']='2020'

fig.show()
fig.write_html(os.path.join(workspace, "Open_Data_Figures/Demographics_County_AgeProfile.html"))

In [ ]:
# get data
dfCensus = census_data
dfAgeSex = dfCensus.loc[dfCensus['variable_category'].isin(['TRPA Census Age Sex Categories Grouped: Age'])]
dfAgeSex_Tahoe = dfAgeSex.loc[dfAgeSex['sample_level']=='tract']
# group by variable name/code/year and sum values
df = dfAgeSex_Tahoe.groupby(['variable_name','variable_code','year_sample'], as_index= False).sum(['value'])

# ordered list to sort age category by
sort_list = [ 'Under 5', '5 to 9','10 to 14', '15 to 19', '20 to 24', 
            '25 to 29', '30 to 34', '35 to 39', '40 to 44', '45 to 49', 
            '50 to 54', '55 to 59', '60 to 64', '65 to 69', '70 to 74', 
            '75 to 79', '80 to 84', 'Over 85']

# setup new fields
# gender is tagged on the end of variable name. split it out after the colon
df['Gender'] = df['variable_name'].str.split(':').str[-1]
# age is the first part of the variable name string
df['Age']    = df['variable_name'].str.split(':').str[0]
# drop the rows that represent the Totals
df = df[df.Age != 'Total']
# setup a new Year column and format it
df['Year'] = pd.to_datetime(df['year_sample'], format='%Y') 
df['Year']   = df['Year'].dt.strftime('%Y')
# setup a new Count field based on the summed values
df['Count'] = df['value']

# cast count as integer type
df = df.astype({'Count':int})

# keep the new columsn
df = df[['Age','Year','Gender','Count']]

# set Age to be a categorical field with the same order as the list
df['Age'] = pd.Categorical(df['Age'], categories = sort_list)
# sort the age category
df = df.sort_values(by='Age')



pivotTable = pd.pivot_table(df, values='Count', index=['Age','Year'],
                       columns=['Gender'], aggfunc=np.sum)
flat = pd.DataFrame(pivotTable.to_records())
df = flat
# remove space in column names 'Male' and "Female"
df.columns = df.columns.str.replace(' ', '')


df['Total_Male'] = df.groupby(['Year'], as_index=False)['Male'].transform('sum')
df['Total_Female'] = df.groupby(['Year'], as_index=False)['Female'].transform('sum')

df['Male_proportion'] = df['Male']/df['Total_Male']
df['Female_proportion'] = df['Female']/df['Total_Female']

df['Male_Percentage'] =  round((df['Male_proportion']*100),2)
df['Female_Percentage'] = round((df['Female_proportion']*100),2)



# Creating instance of the figure
fig = make_subplots(rows=1, cols=4, shared_yaxes=True)

 # Filter by single year
df1990 = df[df['Year']== '1990']
df2000 = df[df['Year']== '2000']
df2010 = df[df['Year']== '2010']
df2020 = df[df['Year']== '2020']

# set x and y variables for plot
y_age90    = df1990['Age']
x_male90   = df1990['Male_proportion']
x_female90 = df1990['Female_proportion'] * -1

# set x and y variables for plot
y_age00    = df2000['Age']
x_male00   = df2000['Male_proportion']
x_female00 = df2000['Female_proportion'] * -1

# set x and y variables for plot
y_age10    = df2010['Age']
x_male10   = df2010['Male_proportion']
x_female10 = df2010['Female_proportion'] * -1

# set x and y variables for plot
y_age20    = df2020['Age']
x_male20   = df2020['Male_proportion']
x_female20 = df2020['Female_proportion'] * -1

mcolor = '#405d74'
fcolor = '#679ab8'
# Adding Male data to the figure
fig.add_trace(go.Bar(y= y_age90, 
					x = x_male90,
					name = 'Male',
					orientation = 'h',
					marker_color=mcolor,
					legendgroup = 'Male',
					customdata = np.stack((df1990['Male_proportion'], df1990['Age']), axis=-1),
					hovertemplate='%{customdata[0]:.1%} of males<extra></extra>'
					),
					row=1, col=1)

# Adding Female data to the figure
fig.add_trace(go.Bar(y = y_age90, 
					x = x_female90,
					name = 'Female', 
					orientation = 'h', 
					marker_color=fcolor,
					legendgroup ='Female',
					customdata = np.stack((df1990['Female_proportion'], df1990['Age']), axis=-1),
					hovertemplate='%{customdata[0]:.1%} of females<extra></extra>'
					),
					row=1, col=1)


# Adding Male data to the figure
fig.add_trace(go.Bar(y= y_age00, 
					x = x_male00,
					name = 'Male',
					orientation = 'h',
					marker_color=mcolor,
					legendgroup = 'Male',
                    showlegend  = False,
					customdata = np.stack((df2000['Male_proportion'], df2000['Age']), axis=-1),
					hovertemplate='%{customdata[0]:.1%} of males<extra></extra>'
					),
					row=1, col=2)

# Adding Female data to the figure
fig.add_trace(go.Bar(y = y_age00, 
					x = x_female00,
					name = 'Female', 
					orientation = 'h', 
					marker_color=fcolor,
					legendgroup ='Female',
                    showlegend  = False,
					customdata = np.stack((df2000['Female_proportion'], df2000['Age']), axis=-1),
					hovertemplate='%{customdata[0]:.1%} of females<extra></extra>'
					),
					row=1, col=2)

# Adding Male data to the figure
fig.add_trace(go.Bar(y= y_age10, 
					x = x_male10,
					name = 'Male',
					orientation = 'h',
					marker_color=mcolor,
					legendgroup = 'Male',
					showlegend  = False,
					customdata = np.stack((df2010['Male_proportion'], df2010['Age']), axis=-1),
					hovertemplate='%{customdata[0]:.1%} of males<extra></extra>'
					),
					row=1, col=3)

# Adding Female data to the figure
fig.add_trace(go.Bar(y = y_age10, 
					x = x_female10,
					name = 'Female', 
					orientation = 'h', 
					marker_color=fcolor,
					legendgroup = 'Female',
					showlegend  = False,
					customdata = np.stack((df2010['Female_proportion'], df2010['Age']), axis=-1),
					hovertemplate='%{customdata[0]:.1%} of females<extra></extra>'
					),
					row=1, col=3)

# Adding Male data to the figure
fig.add_trace(go.Bar(y= y_age20, 
					x = x_male20,
					name = 'Male',
					orientation = 'h',
					marker_color=mcolor,
					legendgroup = 'Male',
					showlegend  = False,
					customdata = np.stack((df2020['Male_proportion'], df2020['Age']), axis=-1),
					hovertemplate='%{customdata[0]:.1%} of males<extra></extra>'
					),
					row=1, col=4)

# Adding Female data to the figure
fig.add_trace(go.Bar(y = y_age20, 
					x = x_female20,
					name = 'Female', 
					orientation = 'h', 
					marker_color=fcolor,
					legendgroup ="Female",
					showlegend  = False,
					customdata = np.stack((df2020['Female_proportion'], df2020['Age']), axis=-1),
					hovertemplate='%{customdata[0]:.1%} of females<extra></extra>'
					),
					row=1, col=4)
axis_format = dict(tickvals = [-.10, -.05, 0,
										.05, .10],
							ticktext = ['10%', '5%', '0', '5%',
										'10%'],
							title_font_size = 14, range=[-.15,.15])

# Updating the layout
fig.update_layout(title = 'Age Profile: Tahoe Basin',
				title_x=0.5,
				title_font_size = 22, barmode = 'relative',
				bargap = 0.0, bargroupgap = 0,
				hovermode="y unified",
				xaxis = axis_format,
				xaxis2 = axis_format,
				xaxis3 = axis_format,
                xaxis4 = axis_format,
				yaxis = dict(					
							title = 'Age',
							title_font_size = 14)			
				)


# edit axis labels
fig['layout']['xaxis']['title'] ='1990'
fig['layout']['xaxis2']['title']='2000'
fig['layout']['xaxis3']['title']='2010'
fig['layout']['xaxis4']['title']='2020'

fig.show()
fig.write_html(os.path.join(workspace, "Open_Data_Figures/Demographics_Tahoe_AgeProfile.html"))

#### Non Tahoe Age Pyramid 5-years

In [ ]:
dfAge = census_data.loc[census_data['variable_category'].isin(['TRPA 10-year Age Categories Grouped: Age'])&(census_data['sample_level'].isin(['tract', 'County']))]
#dfAge_Tahoe = dfAge.loc[dfAgeSex['sample_level']=='tract']
# group by variable name/code/year and sum values
df = dfAge.groupby(['variable_name','variable_code','year_sample', 'sample_level'], as_index= False).sum(['value'])

# ordered list to sort age category by
sort_list = ['Under 10','10 to 19', '20 to 29', '30 to 39', '40 to 49', 
            '50 to 59', '60 to 69', '70 to 79', 'Over 80']

dftest = df

# setup a new Year column and format it
df['Year'] = pd.to_datetime(df['year_sample'], format='%Y') 
df['Year']   = df['Year'].dt.strftime('%Y')
# setup a new Count field based on the summed values
df['Count'] = df['value']
df['Age'] = df['variable_name']
# cast count as integer type
df = df.astype({'Count':int})

# keep the new columsn
df = df[['Age','Year','sample_level','Count']]

# set Age to be a categorical field with the same order as the list
df['Age'] = pd.Categorical(df['Age'], categories = sort_list)
# sort the age category
df = df.sort_values(by='Age')

pivotTable = pd.pivot_table(df, values='Count', index=['Age','Year'],
                       columns=['sample_level'], aggfunc=np.sum)
flat = pd.DataFrame(pivotTable.to_records())
df = flat



# Creating instance of the figure

# Creating instance of the figure
fig = make_subplots(rows=1, cols=4, shared_yaxes=True)

df['Basin']=df['tract'] 
df['Five_County']=df['County']


df['TotalCounty'] = df.groupby(['Year'], as_index=False)['Five_County'].transform('sum')
df['TotalBasin'] = df.groupby(['Year'], as_index=False)['Basin'].transform('sum')

df['Basin_proportion'] = df['Basin']/df['TotalBasin']
df['County_proportion'] = df['County']/df['TotalCounty']

df['Basin_Percentage'] =  round((df['Basin_proportion']*100),2)
df['County_Percentage'] = round((df['County_proportion']*100),2)

 # Filter by single year
df1990 = df[df['Year']== '1990']
df2000 = df[df['Year']== '2000']
df2010 = df[df['Year']== '2010']
df2020 = df[df['Year']== '2020']

# set x and y variables for plot
y_age90    = df1990['Age']
x_basin90   = df1990['Basin_proportion']
x_five_county90 = df1990['County_proportion'] * -1

# set x and y variables for plot
y_age00    = df2000['Age']
x_basin00   = df2000['Basin_proportion']
x_five_county00 = df2000['County_proportion'] * -1

# set x and y variables for plot
y_age10    = df2010['Age']
x_basin10   = df2010['Basin_proportion']
x_five_county10 = df2010['County_proportion'] * -1

# set x and y variables for plot
y_age20    = df2020['Age']
x_basin20   = df2020['Basin_proportion']
x_five_county20 = df2020['County_proportion'] * -1

mcolor = '#405d74'
fcolor = '#e28743'
hovertemplate_percent = '%{customdata[1]}: %{customdata[2]:,.1f}% <extra></extra>'
hovertemplate_basin = 'Basin: %{customdata[2]:,.1f}% <extra></extra>'
hovertemplate_counties = '5-Counties: %{customdata[2]:,.1f}% <extra></extra>'
# Adding Male data to the figure

# Creating instance of the figure
fig = make_subplots(rows=1, cols=4, shared_yaxes=True)
fig.add_trace(go.Bar(y= y_age90, 
					x = x_basin90,
					name = 'Basin',
					orientation = 'h',
					marker_color=mcolor,
					legendgroup = 'Basin',
					customdata = np.stack((df1990['Basin_proportion'], df1990['Age'], df1990['Basin_Percentage']), axis=-1),
					hovertemplate= hovertemplate_basin
					),
					row=1, col=1)

# Adding Female data to the figure
fig.add_trace(go.Bar(y = y_age90, 
					x = x_five_county90,
					name = 'Five County', 
					orientation = 'h', 
					marker_color=fcolor,
					legendgroup ='Five County',
					customdata = np.stack((df1990['County_proportion'], df1990['Age'], df1990['County_Percentage']), axis=-1),
					hovertemplate= hovertemplate_counties
					),
					row=1, col=1)


# Adding Male data to the figure
fig.add_trace(go.Bar(y= y_age00, 
					x = x_basin00,
					name = 'Basin',
					orientation = 'h',
					marker_color=mcolor,
					legendgroup = 'Basin',
                    showlegend  = False,
					customdata = np.stack((df2000['Basin_proportion'], df2000['Age'], df2000['Basin_Percentage']), axis=-1),
					hovertemplate=hovertemplate_basin
					),
					row=1, col=2)

# Adding Female data to the figure
fig.add_trace(go.Bar(y = y_age00, 
					x = x_five_county00,
					name = 'Five County', 
					orientation = 'h', 
					marker_color=fcolor,
					legendgroup ='Five County',
                    showlegend  = False,
					customdata = np.stack((df2000['County_proportion'], df2000['Age'], df2000['County_Percentage']), axis=-1),
					hovertemplate=hovertemplate_counties
					),
					row=1, col=2)

# Adding Male data to the figure
fig.add_trace(go.Bar(y= y_age10, 
					x = x_basin10,
					name = 'Basin',
					orientation = 'h',
					marker_color=mcolor,
					legendgroup = 'Basin',
					showlegend  = False,
					customdata = np.stack((df2010['Basin_proportion'], df2010['Age'], df2010['Basin_Percentage']), axis=-1),
					hovertemplate=hovertemplate_basin
					),
					row=1, col=3)

# Adding Female data to the figure
fig.add_trace(go.Bar(y = y_age10, 
					x = x_five_county10,
					name = 'Five County', 
					orientation = 'h', 
					marker_color=fcolor,
					legendgroup = 'Five County',
					showlegend  = False,
					customdata = np.stack((df2010['County_proportion'], df2010['Age'], df2010['County_Percentage']), axis=-1),
					hovertemplate=hovertemplate_counties
					),
					row=1, col=3)

# Adding Male data to the figure
fig.add_trace(go.Bar(y= y_age20, 
					x = x_basin20,
					name = 'Basin',
					orientation = 'h',
					marker_color=mcolor,
					legendgroup = 'Basin',
					showlegend  = False,
					customdata = np.stack((df2020['Basin_proportion'], df2020['Age'], df2020['Basin_Percentage']), axis=-1),
					hovertemplate=hovertemplate_basin
					),
					row=1, col=4)

# Adding Female data to the figure
fig.add_trace(go.Bar(y = y_age20, 
					x = x_five_county20,
					name = 'Five County', 
					orientation = 'h', 
					marker_color=fcolor,
					legendgroup ="Five County",
					showlegend  = False,
					customdata = np.stack((df2020['County_proportion'], df2020['Age'], df2020['County_Percentage']), axis=-1),
					hovertemplate=hovertemplate_counties
					),
					row=1, col=4)

# Updating the layout
proportion_tick_values = [-.5, -.25, 0, .25, .5]
proportion_tick_text = ['-.5', '-.25', '0', '.25', '.5']
# Updating the layout
fig.update_layout(title = 'Basin vs Five Counties Age Profile: 5 Year Increments',
				title_x=0.5,
				title_font_size = 22, barmode = 'relative',
				bargap = 0.0, bargroupgap = 0,
				hovermode="y unified",
				xaxis = dict(tickvals = proportion_tick_values,
								
							ticktext = proportion_tick_text,
							title_font_size = 14),
				xaxis2 = dict(tickvals = proportion_tick_values,
								
							ticktext = proportion_tick_text,
							title_font_size = 14),
				xaxis3 = dict(tickvals = proportion_tick_values,
								
							ticktext = proportion_tick_text,
							title_font_size = 14),
                xaxis4 = dict(tickvals = proportion_tick_values,
								
							ticktext = proportion_tick_text,
							title_font_size = 14),
				yaxis = dict(					
							title = 'Age',
							title_font_size = 14)			
				)

# edit axis labels
fig['layout']['xaxis']['title'] ='1990 Decennial Census'
fig['layout']['xaxis2']['title']='2000 Decennial Census'
fig['layout']['xaxis3']['title']='2010 Decennial Census'
fig['layout']['xaxis4']['title']='2020 Decennial Census'

fig.show()
fig.write_html(os.path.join(workspace, "Demographics_Tahoe_AgeProfile_5Counties.html")) 

#### Non Tahoe Age Pyramid 10-year

In [ ]:
dfAge = census_data.loc[census_data['variable_category'].isin(['TRPA 10-year Age Categories Grouped: Age'])&(census_data['sample_level'].isin(['tract', 'County']))]
#dfAge_Tahoe = dfAge.loc[dfAgeSex['sample_level']=='tract']
# group by variable name/code/year and sum values
df = dfAge.groupby(['variable_name','variable_code','year_sample', 'sample_level'], as_index= False).sum(['value'])

df_test = df
# ordered list to sort age category by
sort_list = [ 'Under 10',
'10 to 19',
'20 to 29',
'30 to 39',
'40 to 49',
'50 to 59',
'60 to 69',
'70 to 79',
'Over 80'
]


# setup a new Year column and format it
df['Year'] = pd.to_datetime(df['year_sample'], format='%Y') 
df['Year']   = df['Year'].dt.strftime('%Y')
# setup a new Count field based on the summed values
df['Count'] = df['value']
df['Age'] = df['variable_name']
# cast count as integer type
df = df.astype({'Count':int})

# keep the new columsn
df = df[['Age','Year','sample_level','Count']]

# set Age to be a categorical field with the same order as the list
df['Age'] = pd.Categorical(df['Age'], categories = sort_list)
# sort the age category
df = df.sort_values(by='Age')

pivotTable = pd.pivot_table(df, values='Count', index=['Age','Year'],
                       columns=['sample_level'], aggfunc=np.sum)
flat = pd.DataFrame(pivotTable.to_records())
df = flat



# Creating instance of the figure

# Creating instance of the figure
fig = make_subplots(rows=1, cols=4, shared_yaxes=True)

df['Basin']=df['tract'] 
df['Five_County']=df['County']


df['TotalCounty'] = df.groupby(['Year'], as_index=False)['Five_County'].transform('sum')
df['TotalBasin'] = df.groupby(['Year'], as_index=False)['Basin'].transform('sum')

df['Basin_proportion'] = df['Basin']/df['TotalBasin']
df['County_proportion'] = df['County']/df['TotalCounty']

df['Basin_Percentage'] =  round((df['Basin_proportion']*100),2)
df['County_Percentage'] = round((df['County_proportion']*100),2)

 # Filter by single year
df1990 = df[df['Year']== '1990']
df2000 = df[df['Year']== '2000']
df2010 = df[df['Year']== '2010']
df2020 = df[df['Year']== '2020']

# set x and y variables for plot
y_age90    = df1990['Age']
x_basin90   = df1990['Basin_proportion']
x_five_county90 = df1990['County_proportion'] * -1

# set x and y variables for plot
y_age00    = df2000['Age']
x_basin00   = df2000['Basin_proportion']
x_five_county00 = df2000['County_proportion'] * -1

# set x and y variables for plot
y_age10    = df2010['Age']
x_basin10   = df2010['Basin_proportion']
x_five_county10 = df2010['County_proportion'] * -1

# set x and y variables for plot
y_age20    = df2020['Age']
x_basin20   = df2020['Basin_proportion']
x_five_county20 = df2020['County_proportion'] * -1

mcolor = '#405d74'
fcolor = '#e28743'
hovertemplate_percent = '%{customdata[1]}: %{customdata[2]:,.1f}% <extra></extra>'
hovertemplate_basin = 'Basin: %{customdata[2]:,.1f}% <extra></extra>'
hovertemplate_counties = '5-Counties: %{customdata[2]:,.1f}% <extra></extra>'
# Adding Male data to the figure

# Creating instance of the figure
fig = make_subplots(rows=1, cols=4, shared_yaxes=True)
fig.add_trace(go.Bar(y= y_age90, 
					x = x_basin90,
					name = 'Basin',
					orientation = 'h',
					marker_color=mcolor,
					legendgroup = 'Basin',
					customdata = np.stack((df1990['Basin_proportion'], df1990['Age'], df1990['Basin_Percentage']), axis=-1),
					hovertemplate= hovertemplate_basin
					),
					row=1, col=1)

# Adding Female data to the figure
fig.add_trace(go.Bar(y = y_age90, 
					x = x_five_county90,
					name = 'Five County', 
					orientation = 'h', 
					marker_color=fcolor,
					legendgroup ='Five County',
					customdata = np.stack((df1990['County_proportion'], df1990['Age'], df1990['County_Percentage']), axis=-1),
					hovertemplate= hovertemplate_counties
					),
					row=1, col=1)


# Adding Male data to the figure
fig.add_trace(go.Bar(y= y_age00, 
					x = x_basin00,
					name = 'Basin',
					orientation = 'h',
					marker_color=mcolor,
					legendgroup = 'Basin',
                    showlegend  = False,
					customdata = np.stack((df2000['Basin_proportion'], df2000['Age'], df2000['Basin_Percentage']), axis=-1),
					hovertemplate=hovertemplate_basin
					),
					row=1, col=2)

# Adding Female data to the figure
fig.add_trace(go.Bar(y = y_age00, 
					x = x_five_county00,
					name = 'Five County', 
					orientation = 'h', 
					marker_color=fcolor,
					legendgroup ='Five County',
                    showlegend  = False,
					customdata = np.stack((df2000['County_proportion'], df2000['Age'], df2000['County_Percentage']), axis=-1),
					hovertemplate=hovertemplate_counties
					),
					row=1, col=2)

# Adding Male data to the figure
fig.add_trace(go.Bar(y= y_age10, 
					x = x_basin10,
					name = 'Basin',
					orientation = 'h',
					marker_color=mcolor,
					legendgroup = 'Basin',
					showlegend  = False,
					customdata = np.stack((df2010['Basin_proportion'], df2010['Age'], df2010['Basin_Percentage']), axis=-1),
					hovertemplate=hovertemplate_basin
					),
					row=1, col=3)

# Adding Female data to the figure
fig.add_trace(go.Bar(y = y_age10, 
					x = x_five_county10,
					name = 'Five County', 
					orientation = 'h', 
					marker_color=fcolor,
					legendgroup = 'Five County',
					showlegend  = False,
					customdata = np.stack((df2010['County_proportion'], df2010['Age'], df2010['County_Percentage']), axis=-1),
					hovertemplate=hovertemplate_counties
					),
					row=1, col=3)

# Adding Male data to the figure
fig.add_trace(go.Bar(y= y_age20, 
					x = x_basin20,
					name = 'Basin',
					orientation = 'h',
					marker_color=mcolor,
					legendgroup = 'Basin',
					showlegend  = False,
					customdata = np.stack((df2020['Basin_proportion'], df2020['Age'], df2020['Basin_Percentage']), axis=-1),
					hovertemplate=hovertemplate_basin
					),
					row=1, col=4)

# Adding Female data to the figure
fig.add_trace(go.Bar(y = y_age20, 
					x = x_five_county20,
					name = 'Five County', 
					orientation = 'h', 
					marker_color=fcolor,
					legendgroup ="Five County",
					showlegend  = False,
					customdata = np.stack((df2020['County_proportion'], df2020['Age'], df2020['County_Percentage']), axis=-1),
					hovertemplate=hovertemplate_counties
					),
					row=1, col=4)

# Updating the layout
proportion_tick_values = [-.5, -.25, 0, .25, .5]
proportion_tick_text = ['-.5', '-.25', '0', '.25', '.5']
# Updating the layout
fig.update_layout(title = 'Basin vs Five Counties Age Profile: 5 Year Increments',
				title_x=0.5,
				title_font_size = 22, barmode = 'relative',
				bargap = 0.0, bargroupgap = 0,
				hovermode="y unified",
				xaxis = dict(tickvals = proportion_tick_values,
								
							ticktext = proportion_tick_text,
							title_font_size = 14),
				xaxis2 = dict(tickvals = proportion_tick_values,
								
							ticktext = proportion_tick_text,
							title_font_size = 14),
				xaxis3 = dict(tickvals = proportion_tick_values,
								
							ticktext = proportion_tick_text,
							title_font_size = 14),
                xaxis4 = dict(tickvals = proportion_tick_values,
								
							ticktext = proportion_tick_text,
							title_font_size = 14),
				yaxis = dict(					
							title = 'Age',
							title_font_size = 14)			
				)

# edit axis labels
fig['layout']['xaxis']['title'] ='1990 Decennial Census'
fig['layout']['xaxis2']['title']='2000 Decennial Census'
fig['layout']['xaxis3']['title']='2010 Decennial Census'
fig['layout']['xaxis4']['title']='2020 Decennial Census'

fig.show()
fig.write_html(os.path.join(workspace, "Demographics_Tahoe_AgeProfile_5Counties.html")) 

#### Grouped Age Chart

In [ ]:
dfAge = census_data.loc[census_data['variable_category'].isin(['TRPA Census Age Categories Grouped: Age'])&(census_data['sample_level'].isin(['tract', 'County']))]
dfAge = dfAge.loc[dfAge['variable_name']!='Total']
#dfAge_Tahoe = dfAge.loc[dfAgeSex['sample_level']=='tract']
# group by variable name/code/year and sum values
df = dfAge.groupby(['variable_name','variable_code','year_sample', 'sample_level'], as_index= False).sum(['value'])
# ordered list to sort age category by
over_65_list = ['65 to 69', '70 to 74', 
            '75 to 79', '80 to 84', 'Over 85']
df['Over_65'] = np.where(df['variable_name'].isin(over_65_list), 'Yes', 'No')

df['Total_Population'] = df.groupby(['year_sample', 'sample_level'])['value'].transform('sum')

df['Total_Over_65'] = df.groupby(['year_sample', 'sample_level', 'Over_65'])['value'].transform('sum')

df = df.groupby(['year_sample', 'sample_level', 'Over_65', 'Total_Population', 'Total_Over_65']).count().reset_index()

df = df.sort_values(by=['year_sample', 'sample_level'], ascending=[True, False], ignore_index=True)

df_graph = df[['year_sample', 'sample_level', 'Over_65', 'Total_Population', 'Total_Over_65']]

df_graph = df_graph.loc[df_graph['Over_65']=='Yes']

df_graph['Percent_Over_65'] = df_graph['Total_Over_65']/df_graph['Total_Population']

#df_graph = df_graph.loc[df_graph['sample_level']=='tract']
df_graph['Geography'] = np.where(df_graph['sample_level']=='tract', 'Tahoe Basin', 'Surrounding 5-Counties')

color_mapping = {
    "Tahoe Basin": '#00407a',
    'Surrounding 5-Counties': '#996308'
}

fig = px.bar(df_graph, x='year_sample', 
             y="Percent_Over_65", barmode='group', title = "Percent of Population over 65", color='Geography',
              opacity = .90, color_discrete_map= color_mapping
             )
fig.update_layout(font_family=font,
                 template=template,
                 bargroupgap = 0.20,
                    hoverlabel=dict(
                    bgcolor='white',
                       font=dict(color='#6680A8') 
                    ),
    legend=dict(
        title='',  # Legend title
        
       
        font=dict(size=16)
    ),
                     title=dict(
                                x=0.5,
                                xanchor='center',
                                font=dict(size=32)),
                     xaxis=dict(
                                tick0=1990, 
                                dtick=10,
                                title = 'Census Year',
							                  title_font_size = 24,
                                tickfont = dict(size=20)),
                     yaxis=dict(
                                title = '',
							                  title_font_size = 24,
                                tickfont = dict(size=20),
                                tickformat= ',.0%'))
fig.update_traces(hovertemplate='Census Year: %{x} <br> Percent over 65: %{y:.1%} <extra></extra>', width = 3.5)
fig.show()
fig.write_html(os.path.join(workspace, "Open_Data_Figures/Demographics_Tahoe_Senior_Population.html")) 

In [ ]:
# get data
dfCensus = pd.read_sql("SELECT * FROM sde.SDE.Census_Demographics", conn)
dfAgeSex = dfCensus.loc[dfCensus['variable_category'].isin(['TRPA Census Age Sex Categories Grouped: Age'])]
dfAgeSex_Tahoe = dfAgeSex.loc[dfAgeSex['sample_level']=='tract']
dfAgeSex_5_County = dfAgeSex.loc[dfAgeSex['sample_level']=='county']
# group by variable name/code/year and sum values
df = dfAgeSex_Tahoe.groupby(['variable_name','variable_code','year_sample'], as_index= False).sum(['value'])
#df= dfAgeSex_5_County.groupby(['variable_name','variable_code','year_sample'], as_index= False).sum(['value'])
# ordered list to sort age category by
sort_list = [ 'Under 5', '5 to 9','10 to 14', '15 to 19', '20 to 24', 
            '25 to 29', '30 to 34', '35 to 39', '40 to 44', '45 to 49', 
            '50 to 54', '55 to 59', '60 to 64', '65 to 69', '70 to 74', 
            '75 to 79', '80 to 84', 'Over 85']

# setup new fields
# gender is tagged on the end of variable name. split it out after the colon
df['Gender'] = df['variable_name'].str.split(':').str[-1]
# age is the first part of the variable name string
df['Age']    = df['variable_name'].str.split(':').str[0]
# drop the rows that represent the Totals
df = df[df.Age != 'Total']
# setup a new Year column and format it
df['Year'] = pd.to_datetime(df['year_sample'], format='%Y') 
df['Year']   = df['Year'].dt.strftime('%Y')
# setup a new Count field based on the summed values
df['Count'] = df['value']

# cast count as integer type
df = df.astype({'Count':int})

# keep the new columsn
df = df[['Age','Year','Gender','Count']]

# set Age to be a categorical field with the same order as the list
df['Age'] = pd.Categorical(df['Age'], categories = sort_list)
# sort the age category
df = df.sort_values(by='Age')

pivotTable = pd.pivot_table(df, values='Count', index=['Age','Year'],
                       columns=['Gender'], aggfunc=np.sum)
flat = pd.DataFrame(pivotTable.to_records())
df = flat
# remove space in column names 'Male' and "Female"
df.columns = df.columns.str.replace(' ', '')

df['Total_Population'] = df['Female']+df['Male']
df['TotalSum'] = df.groupby(['Year'], as_index=False)['Total_Population'].transform('sum')
df['Female_proportion'] = df['Female']/df['TotalSum']
df['Male_proportion'] = df['Male']/df['TotalSum']

# Creating instance of the figure
fig = make_subplots(rows=1, cols=4, shared_yaxes=True)

 # Filter by single year
df1990 = df[df['Year']== '1990']
df2000 = df[df['Year']== '2000']
df2010 = df[df['Year']== '2010']
df2020 = df[df['Year']== '2020']

# set x and y variables for plot
y_age90    = df1990['Age']
x_male90   = df1990['Male_proportion']
x_female90 = df1990['Female_proportion'] * -1

# set x and y variables for plot
y_age00    = df2000['Age']
x_male00   = df2000['Male_proportion']
x_female00 = df2000['Female_proportion'] * -1

# set x and y variables for plot
y_age10    = df2010['Age']
x_male10   = df2010['Male_proportion']
x_female10 = df2010['Female_proportion'] * -1

# set x and y variables for plot
y_age20    = df2020['Age']
x_male20   = df2020['Male_proportion']
x_female20 = df2020['Female_proportion'] * -1

mcolor = '#405d74'
fcolor = '#679ab8'
# Adding Male data to the figure
fig.add_trace(go.Bar(y= y_age90, 
					x = x_male90,
					name = 'Male',
					orientation = 'h',
					marker_color=mcolor,
					legendgroup = 'Male',
					customdata = np.stack((df1990['Male_proportion'], df1990['Age']), axis=-1),
					hovertemplate='%{customdata[0]:,.0f} males<extra></extra>'
					),
					row=1, col=1)

# Adding Female data to the figure
fig.add_trace(go.Bar(y = y_age90, 
					x = x_female90,
					name = 'Female', 
					orientation = 'h', 
					marker_color=fcolor,
					legendgroup ='Female',
					customdata = np.stack((df1990['Female_proportion'], df1990['Age']), axis=-1),
					hovertemplate='%{customdata[0]:,.0f} females<extra></extra>'
					),
					row=1, col=1)


# Adding Male data to the figure
fig.add_trace(go.Bar(y= y_age00, 
					x = x_male00,
					name = 'Male',
					orientation = 'h',
					marker_color=mcolor,
					legendgroup = 'Male',
                    showlegend  = False,
					customdata = np.stack((df2000['Male_proportion'], df2000['Age']), axis=-1),
					hovertemplate='%{customdata[0]:,.0f} males<extra></extra>'
					),
					row=1, col=2)

# Adding Female data to the figure
fig.add_trace(go.Bar(y = y_age00, 
					x = x_female00,
					name = 'Female', 
					orientation = 'h', 
					marker_color=fcolor,
					legendgroup ='Female',
                    showlegend  = False,
					customdata = np.stack((df2000['Female_proportion'], df2000['Age']), axis=-1),
					hovertemplate='%{customdata[0]:,.0f} females<extra></extra>'
					),
					row=1, col=2)

# Adding Male data to the figure
fig.add_trace(go.Bar(y= y_age10, 
					x = x_male10,
					name = 'Male',
					orientation = 'h',
					marker_color=mcolor,
					legendgroup = 'Male',
					showlegend  = False,
					customdata = np.stack((df2010['Male_proportion'], df2010['Age']), axis=-1),
					hovertemplate='%{customdata[0]:,.0f} males<extra></extra>'
					),
					row=1, col=3)

# Adding Female data to the figure
fig.add_trace(go.Bar(y = y_age10, 
					x = x_female10,
					name = 'Female', 
					orientation = 'h', 
					marker_color=fcolor,
					legendgroup = 'Female',
					showlegend  = False,
					customdata = np.stack((df2010['Female_proportion'], df2010['Age']), axis=-1),
					hovertemplate='%{customdata[0]:,.0f} females<extra></extra>'
					),
					row=1, col=3)

# Adding Male data to the figure
fig.add_trace(go.Bar(y= y_age20, 
					x = x_male20,
					name = 'Male',
					orientation = 'h',
					marker_color=mcolor,
					legendgroup = 'Male',
					showlegend  = False,
					customdata = np.stack((df2020['Male_proportion'], df2020['Age']), axis=-1),
					hovertemplate='%{customdata[0]:,.0f} males<extra></extra>'
					),
					row=1, col=4)

# Adding Female data to the figure
fig.add_trace(go.Bar(y = y_age20, 
					x = x_female20,
					name = 'Female', 
					orientation = 'h', 
					marker_color=fcolor,
					legendgroup ="Female",
					showlegend  = False,
					customdata = np.stack((df2020['Female_proportion'], df2020['Age']), axis=-1),
					hovertemplate='%{customdata[0]:,.0f} females<extra></extra>'
					),
					row=1, col=4)

proportion_tick_values = [-.5, -.25, 0, .25, .5]
proportion_tick_text = ['-.5', '-.25', '0', '.25', '.5']
# Updating the layout
fig.update_layout(title = 'Tahoe Basin Age Profile: 5 Year Increments',
				title_x=0.5,
				title_font_size = 22, barmode = 'relative',
				bargap = 0.0, bargroupgap = 0,
				hovermode="y unified",
				xaxis = dict(tickvals = proportion_tick_values,
								
							ticktext = proportion_tick_text,
							title_font_size = 14),
				xaxis2 = dict(tickvals = proportion_tick_values,
								
							ticktext = proportion_tick_text,
							title_font_size = 14),
				xaxis3 = dict(tickvals = proportion_tick_values,
								
							ticktext = proportion_tick_text,
							title_font_size = 14),
                xaxis4 = dict(tickvals = proportion_tick_values,
								
							ticktext = proportion_tick_text,
							title_font_size = 14),
				yaxis = dict(					
							title = 'Age',
							title_font_size = 14)			
				)


# edit axis labels
fig['layout']['xaxis']['title'] ='1990 Decennial Census'
fig['layout']['xaxis2']['title']='2000 Decennial Census'
fig['layout']['xaxis3']['title']='2010 Decennial Census'
fig['layout']['xaxis4']['title']='2020 Decennial Census'

fig.show()
fig.write_html(os.path.join(workspace, "Demographics_Tahoe_AgeProfile_DecadeSubplots_proportion.html"))

### Race

In [ ]:
## fix colors
dfCensus = census_data
dfRace = dfCensus.loc[dfCensus.variable_category.isin(['Race'])]
dfRace = dfRace.loc[dfRace['year_sample']!=1990]
dfRace = dfRace[dfRace.dataset != 'acs/acs5']
# group by variable name/code/year and sum values
df = dfRace.groupby(['variable_name','variable_code','year_sample'], as_index= False).sum(['value'])

# setup new fields
# fix text of race names and filter out non-hispanic so we get a true total of all races
df['Race'] = df['variable_name'].str.split(':').str[-1]
# strips off the prefix
df.loc[df['Race'].str.split(';').str[0] == '  Not Hispanic or Latino', 'Race'] =  df['Race'].str.split(';').str[-1]
# get rid of spaces
df.Race = df.Race.str.strip()
# drop the not hispanic total rows
df = df[df.Race != 'Not Hispanic or Latino']
# setup a new Year column and format it
df['Year'] = pd.to_datetime(df['year_sample'], format='%Y') 
df['Year']   = df['Year'].dt.strftime('%Y')
# setup a new Count field based on the summed values
df['Count'] = df['value']
# cast count as integer type
df = df.astype({'Count':int})
df['Percent'] = 0.0
# keep the new columns
df = df[['Race','Year','Percent','Count']]

df00 = df[df['Year']== '2000']
df10 = df[df['Year']== '2010']
df20 = df[df['Year']== '2020']
# df21 = df[df['Year']== '2021']

# calculate percents of total for each year
df.loc[df['Year'] == '2000', 'Percent'] =  (df00['Count'] / df00['Count'].sum()) * 100
df.loc[df['Year'] == '2010', 'Percent'] =  (df10['Count'] / df10['Count'].sum()) * 100
df.loc[df['Year'] == '2020', 'Percent'] =  (df20['Count'] / df20['Count'].sum()) * 100
# df.loc[df['Year'] == '2021', 'Percent'] =  (df21['Count'] / df21['Count'].sum()) * 100

color_mapping = {
    "American Indian/Alaskan Native" : "#ff7fe9",
"Asian" : " #32ef94",
"Black or African American" : "#00b6f1",
"Hispanic or Latino" : "#e8ca0d",
"Pacific Islander/Hawaiian Native" : "#e2c4a5",
"Other race" : "#ff6a00",
"Two or more races" : "#96f7ef",
"White (non-Hispanic)" : "#f23c3f"
}

category_sort = ['White (non-Hispanic)',
 'Hispanic or Latino',
 'Black or African American',
 'Asian',
 'American Indian/Alaskan Native',
 'Pacific Islander/Hawaiian Native',
 'Other race',
 'Two or more races']

label_updates = {'American Indian and Alaska Native alone' : "American Indian/Alaskan Native",
'Asian alone' : "Asian",
'Black or African American alone' : "Black or African American",
'Hispanic or Latino' : "Hispanic or Latino",
'Native Hawaiian and Other Pacific Islander alone' : "Pacific Islander/Hawaiian Native",
'Some other race alone' : "Other race",
'Two or more races' : "Two or more races",
'White alone' : "White (non-Hispanic)",
}

df['Race_Label']=df['Race'].map(label_updates)
# Sort the DataFrame based on the custom order of the 'Name' column
df = df.sort_values(by=['Race_Label'], key=lambda x: x.map({v: i for i, v in enumerate(category_sort)}))
df.reset_index(drop=True, inplace=True)
pivotRace = pd.pivot_table(df,index=['Year'],
                              columns='Race_Label',
                              values='Percent')

flat = pd.DataFrame(pivotRace.to_records())
flat_columns = ['Year'] + category_sort
flat = flat[flat_columns]
df = flat

raceList = df.columns.to_list()
raceList.remove('Year')
fig = px.bar(df, x="Year", 
             y=raceList,
             color_discrete_map=color_mapping, opacity=.9,title='Race and Hispanic Origin in Tahoe Basin'
             )

fig.update_traces(hovertemplate='<br> %{y:.2f}%')

# set layout

fig.update_layout(
                    font_family=font,
                    template=template,
                    showlegend=True,
                    legend_title='',
                    hovermode="x unified",
                    title=dict(
                                x=0.5,
                                xanchor='center',
                                font=dict(size=32)),
                    xaxis = dict(
                        type='category',
                        tickmode = 'linear',
							                  title_font_size = 24,
                                tickfont = dict(size=20)
                    ),
                    yaxis = dict(
                        autorange = "reversed",
                        tickmode = 'array',
                        tick0 = 0,
                        dtick = 20,
                        range=[0, 100],
                        tickvals = [0, 20, 40, 60, 80, 100],
                        ticktext = ['100', '80', '60', '40', '20', '0'],
                        title_text='Percent',
							                  title_font_size = 24,
                                tickfont = dict(size=20)
                    )
                 )

fig.show()

In [ ]:

fig.write_html(os.path.join(workspace, "Open_Data_Figures\Demographics_Tahoe_RaceDistribution.html"))

### Population change by neighborhood

In [ ]:
service_url = 'https://maps.trpa.org/server/rest/services/Demographics/FeatureServer/31'

feature_layer = FeatureLayer(service_url)
query_result = feature_layer.query()
# Convert the query result to a list of dictionaries
feature_list = query_result.features

# Create a pandas DataFrame from the list of dictionaries
neighborhood_data = pd.DataFrame([feature.attributes for feature in feature_list])


In [ ]:
neighborhood_pop_change = neighborhood_data[['NEIGHBORHOOD', 'STATE', 'Population_Change']]

neighborhood_pop_change.sort_values(by=['STATE','NEIGHBORHOOD'], inplace=True, ascending=False, ignore_index = True)

color_mapping = {
    'NV': '#ed954d',
    'CA': '#07467c'
}


'''def conditional_color(row):
    if row['STATE'] == 'CA' and row['Population_Change']>=0:
        return '#0a4c86'
    if row['STATE'] == 'CA' and row['Population_Change']<0:
        return '#4fb5ce'
    if row['STATE'] == 'NV' and row['Population_Change']>=0:
        return '#f87e3b'
    if row['STATE'] == 'NV' and row['Population_Change']<0:
        return '#f0d087'


neighborhood_pop_change['Color']= neighborhood_pop_change.apply(conditional_color, axis=1)
'''

fig = px.bar(neighborhood_pop_change, x='Population_Change', 
             y="NEIGHBORHOOD", color= 'STATE', 
            color_discrete_map=color_mapping
             )
#fig.update_traces(marker_color=neighborhood_pop_change["Color"])
fig.update_layout(
    title='Population Change by Community: 2010 to 2020',
    title_x= 0.5,
    title_font = dict(size=24),
    xaxis_title='Population Change',
    yaxis_title='Community',
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(
        title='State',  # Legend title
        
        x=0.85,  # X-position of the legend
        y=1,  # Y-position of the legend
        traceorder = 'reversed',
        font=dict(size=16)
    )
    #legend_title='State',
    #legend_traceorder='reversed'
)

fig.update_xaxes(
    range=[-800, 800]  # Set the x-axis range
)
fig.show()

fig.write_html(os.path.join(workspace, "Demographics_Tahoe_Population_Change_Neighborhood.html"))


### Color Adjusted

In [ ]:
neighborhood_pop_change = neighborhood_data[['NEIGHBORHOOD', 'STATE', 'Population_Change']]

neighborhood_pop_change.sort_values(by=['STATE','NEIGHBORHOOD'], inplace=True, ascending=False, ignore_index = True)

color_mapping = {
    'NV': '#ed954d',
    'CA': '#07467c'
}


color_mapping_pos = {
    'NV_Pos': '#ed954d',
    'CA_Pos': '#07467c',
    'NV_Neg': '#f3db9d',
    'CA_Neg': '#87d0df',

}

def conditional_color(row):
    if row['STATE'] == 'CA' and row['Population_Change']>=0:
        return 'CA_Pos'
    if row['STATE'] == 'CA' and row['Population_Change']<0:
        return 'CA_Neg'
    if row['STATE'] == 'NV' and row['Population_Change']>=0:
        return 'NV_Pos'
    if row['STATE'] == 'NV' and row['Population_Change']<0:
        return '#NV_Neg'


neighborhood_pop_change['Color']= neighborhood_pop_change.apply(conditional_color, axis=1)
neighborhood_pop_change['Alpha'] = np.where(neighborhood_pop_change['Population_Change'] >= 0, 1, 0.3)
'''
fig = px.bar(neighborhood_pop_change, x='Population_Change', 
             y="NEIGHBORHOOD", color= 'STATE',
            color_discrete_map=color_mapping
             )
'''
'''
for index, row in neighborhood_pop_change.iterrows():
    alpha = row['Alpha']
    print(index)
    print(alpha)
    fig.update_traces(marker=dict(opacity=alpha), selector=dict(index=index))
'''
'''
fig.update_traces(marker=dict(color='#440900'), selector=dict(index=4))

#fig.update_traces(marker_color=neighborhood_pop_change["Color"])
fig.update_layout(
    title='Population Change by Community: 2010 to 2020',
    title_x= 0.5,
    title_font = dict(size=24),
    xaxis_title='Population Change',
    yaxis_title='Community',
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(
        title='State',  # Legend title
        
        x=0.85,  # X-position of the legend
        y=1,  # Y-position of the legend
        traceorder = 'reversed',
        font=dict(size=16)
    )
    #legend_title='State',
    #legend_traceorder='reversed'
)

fig.update_xaxes(
    range=[-800, 800]  # Set the x-axis range
)
fig.show()
for i, trace in enumerate(fig.data):
    print(f"Trace {i} index: {trace.index}")
#fig.write_html(os.path.join(workspace, "Demographics_Tahoe_Population_Change_Neighborhood.html"))


'''
fig = go.Figure()

legend_added = set()  # To track added legends

for index, row in neighborhood_pop_change.iterrows():
    color = color_mapping_pos.get(row['Color'], 'gray')  # Default to gray if no mapping
    state_color  = color_mapping.get(row['STATE'], 'gray')
    if row['STATE'] not in legend_added:
        legend_added.add(row['STATE'])  # Add to the set of added legends
        
        fig.add_trace(go.Bar(y=[row['NEIGHBORHOOD']], x=[0], orientation='h', marker_color=state_color,
                             opacity=1, name=row['STATE'], legendgroup=row['STATE']))
        
    fig.add_trace(go.Bar(y=[row['NEIGHBORHOOD']], x=[row['Population_Change']], orientation='h', marker_color=color,
                          # Use alpha column for transparency in traces,
                         legendgroup=row['STATE'], showlegend=False))

fig.update_layout(
    title='Population Change by Community: 2010 to 2020',
    title_x= 0.5,
    barmode = 'relative',
    title_font = dict(size=24),
    xaxis_title='Population Change',
    yaxis_title='Community',
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(
        title='State',  # Legend title
        
        x=0.85,  # X-position of the legend
        y=1,  # Y-position of the legend
        traceorder = 'reversed',
        font=dict(size=16)
    )
    #legend_title='State',
    #legend_traceorder='reversed'
)

fig.update_xaxes(
    range=[-800, 800]  # Set the x-axis range
)
fig.show()

fig.write_html(os.path.join(workspace, "Demographics_Tahoe_Population_Change_Neighborhood_colormod.html"))


### Alpha adjusted

In [ ]:
neighborhood_pop_change = neighborhood_data[['NEIGHBORHOOD', 'STATE', 'Population_Change']]

neighborhood_pop_change.sort_values(by=['STATE','NEIGHBORHOOD'], inplace=True, ascending=False, ignore_index = True)


color_mapping = {
    'NV': '#ed954d',
    'CA': '#07467c'
}


color_mapping_pos = {
    'NV_Pos': '#ed954d',
    'CA_Pos': '#07467c',
    'NV_Neg': '#f3db9d',
    'CA_Neg': '#87d0df',

}

def conditional_color(row):
    if row['STATE'] == 'CA' and row['Population_Change']>=0:
        return 'CA_Pos'
    if row['STATE'] == 'CA' and row['Population_Change']<0:
        return 'CA_Neg'
    if row['STATE'] == 'NV' and row['Population_Change']>=0:
        return 'NV_Pos'
    if row['STATE'] == 'NV' and row['Population_Change']<0:
        return '#NV_Neg'


neighborhood_pop_change['Color']= neighborhood_pop_change.apply(conditional_color, axis=1)
neighborhood_pop_change['Alpha'] = np.where(neighborhood_pop_change['Population_Change'] >= 0, 1, 0.6)

fig = go.Figure()

legend_added = set()  # To track added legends

for index, row in neighborhood_pop_change.iterrows():
    color =color_mapping.get(row['STATE'], 'gray') # Default to gray if no mapping
    state_color  = color_mapping.get(row['STATE'], 'gray')
    if row['STATE'] not in legend_added:
        legend_added.add(row['STATE'])  # Add to the set of added legends
        
        fig.add_trace(go.Bar(y=[row['NEIGHBORHOOD']], x=[0], orientation='h', marker_color=state_color,
                             opacity=1, name=row['STATE'], legendgroup=row['STATE']))
        
    fig.add_trace(go.Bar(y=[row['NEIGHBORHOOD']], x=[row['Population_Change']], orientation='h', marker_color=color,
                          # Use alpha column for transparency in traces,
                          opacity=row['Alpha'], 
                         legendgroup=row['STATE'], showlegend=False))

fig.update_layout(
    title='Population Change by Community: 2010 to 2020',
    title_x= 0.5,
    barmode = 'relative',
    title_font = dict(size=24),
    xaxis_title='Population Change',
    yaxis_title='',
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(
        title='State',  # Legend title
        
        x=0.85,  # X-position of the legend
        y=1,  # Y-position of the legend
        traceorder = 'reversed',
        font=dict(size=16)
    )
    #legend_title='State',
    #legend_traceorder='reversed'
)

fig.update_xaxes(
    range=[-800, 800], tickfont = dict(size=20)  # Set the x-axis range
)

fig.update_yaxes(tickfont = dict(size=16))

fig.update_traces(hovertemplate='Community: %{y} <br> Population Change: %{x:,.0f}<extra></extra>')

fig.show()

fig.write_html(os.path.join(workspace, "Open_Data_Figures/Demographics_Tahoe_Population_Change_Neighborhood_alphamod.html"))


In [ ]:
neighborhood_housing_change = neighborhood_data[['NEIGHBORHOOD', 'STATE', 'Housing_Unit_Change']]

neighborhood_housing_change.sort_values(by=['STATE','NEIGHBORHOOD'], inplace=True, ascending=False)


color_mapping = {
    'NV': '#ed954d',
    'CA': '#07467c'
}

neighborhood_housing_change['Alpha'] = np.where(neighborhood_housing_change['Housing_Unit_Change'] >= 0, 1, 0.6)

fig = go.Figure()

legend_added = set()  # To track added legends

for index, row in neighborhood_housing_change.iterrows():
    color =color_mapping.get(row['STATE'], 'gray') # Default to gray if no mapping
    state_color  = color_mapping.get(row['STATE'], 'gray')
    if row['STATE'] not in legend_added:
        legend_added.add(row['STATE'])  # Add to the set of added legends
        
        fig.add_trace(go.Bar(y=[row['NEIGHBORHOOD']], x=[0], orientation='h', marker_color=state_color,
                             opacity=1, name=row['STATE'], legendgroup=row['STATE']))
        
    fig.add_trace(go.Bar(y=[row['NEIGHBORHOOD']], x=[row['Housing_Unit_Change']], orientation='h', marker_color=color,
                          # Use alpha column for transparency in traces,
                          opacity=row['Alpha'], 
                         legendgroup=row['STATE'], showlegend=False))

fig.update_layout(
    title='Housing Unit Change by Community: 2010 to 2020',
    title_x= 0.5,
    barmode = 'relative',
    title_font = dict(size=24),
    xaxis_title='Housing Unit Change',
    yaxis_title='Community',
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(
        title='State',  # Legend title
        
        x=0.85,  # X-position of the legend
        y=1,  # Y-position of the legend
        traceorder = 'reversed',
        font=dict(size=16)
    )
    #legend_title='State',
    #legend_traceorder='reversed'
)

fig.update_xaxes(
    range=[-1000, 1000]  # Set the x-axis range
)
fig.show()

fig.write_html(os.path.join(workspace, "Open_Data_Figures\Demographics_Tahoe_Housing_Change_Neighborhood_alphamod.html"))



In [ ]:
neighborhood_housing_change = neighborhood_data[['NEIGHBORHOOD', 'STATE', 'Housing_Unit_Change']]

neighborhood_housing_change.sort_values(by=['STATE','NEIGHBORHOOD'], inplace=True, ascending=False)


color_mapping = {
    'NV': '#ed954d',
    'CA': '#07467c'
}




fig = px.bar(neighborhood_housing_change, x='Housing_Unit_Change', 
             y="NEIGHBORHOOD", color= "STATE",
             color_discrete_map=color_mapping
             )

fig.update_layout(
    title='Housing Unit Change by Community: 2010 to 2020',
    title_x= 0.5,
    title_font = dict(size=24),
    xaxis_title='Housing Unit Change',
    yaxis_title='Community',
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(
        title='State',  # Legend title
        
        x=0.85,  # X-position of the legend
        y=1,  # Y-position of the legend
        traceorder = 'reversed',
        font=dict(size=16)
    )
    #legend_title='State',
    #legend_traceorder='reversed'
)

fig.update_xaxes(
    range=[-1000, 1000]  # Set the x-axis range
)
fig.show()

fig.write_html(os.path.join(workspace, "Open_Data_Figures\Demographics_Tahoe_Housing_Change_Neighborhood.html"))


### Population and Housing Unit by neighborhood on the same figure

In [ ]:
#Not working right now
fig = make_subplots(rows=2, cols=1, shared_yaxes=True)

fig.add_trace(go.Bar(y = neighborhood_pop_change['NEIGHBORHOOD'], 
					x = neighborhood_pop_change['Population_Change']
             ),
			 row=1, col=1)

"""fig.add_trace(go.Bar(neighborhood_housing_change, x='Housing_Unit_Change', 
             y="NEIGHBORHOOD", color= "STATE",
             color_discrete_map=color_mapping
             ),
					row=2, col=1)"""
fig.show()

### Housing

In [ ]:
## get housing unit data
service_url = 'https://maps.trpa.org/server/rest/services/Demographics/FeatureServer/28'

feature_layer = FeatureLayer(service_url)
query_result = feature_layer.query()
# Convert the query result to a list of dictionaries
feature_list = query_result.features

# Create a pandas DataFrame from the list of dictionaries
all_data = pd.DataFrame([feature.attributes for feature in feature_list])

### Population by State, County, and Selected Neighborhoods for Census Maps

In [ ]:
service_url = 'https://maps.trpa.org/server/rest/services/Demographics/FeatureServer/31'

feature_layer = FeatureLayer(service_url)
query_result = feature_layer.query()
# Convert the query result to a list of dictionaries
feature_list = query_result.features

# Create a pandas DataFrame from the list of dictionaries
neighborhood_data = pd.DataFrame([feature.attributes for feature in feature_list])

In [ ]:
dfHousing = census_data.loc[(census_data.variable_category.isin(['Housing Units']))&(census_data['sample_level']=='tract')&(census_data['county']!='510')]
dfHousing = dfHousing[dfHousing.dataset != 'acs/acs5']

# group the data
df = dfHousing.groupby(['variable_name', 'year_sample'], as_index = False).sum(['value'])

# Define the new row to be added
new_row = {'variable_name': 'Vacant Housing Units: Other', 'year_sample': 2020, 'value': 0}
# Use the loc method to add the new row to the DataFrame
df.loc[len(df)] = new_row
# Define the new row to be added
new_row = {'variable_name': 'Vacant Housing Units: Other', 'year_sample': 2010, 'value': 0}
# Use the loc method to add the new row to the DataFrame
df.loc[len(df)] = new_row
# Define the new row to be added
new_row = {'variable_name': 'Vacant Housing Units: Other', 'year_sample': 2000, 'value': 0}
# Use the loc method to add the new row to the DataFrame
df.loc[len(df)] = new_row

new_row = {'variable_name': 'Vacant Housing Units: Other', 'year_sample': 1990, 'value': 0}
# Use the loc method to add the new row to the DataFrame
df.loc[len(df)] = new_row

# Define conditions for rows
year1990 = df['year_sample'] == 1990
year2000 = df['year_sample'] == 2000
year2010 = df['year_sample'] == 2010
year2020 = df['year_sample'] == 2020

condition1 = df['variable_name'] == 'Total Housing Units: Vacant'
condition2 = df['variable_name'] == 'Vacant Housing Units: Seasonal, recreational, or occasional use'
target_row = df['variable_name'] == 'Vacant Housing Units: Other'

# Calculate the difference between rows based on conditions

diff_values90 = df.loc[year1990&condition1, 'value'].reset_index(drop=True) - df.loc[year1990&condition2, 'value'].reset_index(drop=True)
diff_values00 = df.loc[year2000&condition1, 'value'].reset_index(drop=True) - df.loc[year2000&condition2, 'value'].reset_index(drop=True)
diff_values10 = df.loc[year2010&condition1, 'value'].reset_index(drop=True) - df.loc[year2010&condition2, 'value'].reset_index(drop=True)
diff_values20 = df.loc[year2020&condition1, 'value'].reset_index(drop=True) - df.loc[year2020&condition2, 'value'].reset_index(drop=True)

# Update the 'value' column for rows with the difference values for each year
df.loc[target_row&year1990, 'value'] = diff_values90[0]
df.loc[target_row&year2000, 'value'] = diff_values00[0]
df.loc[target_row&year2010, 'value'] = diff_values10[0]
df.loc[target_row&year2020, 'value'] = diff_values20[0]

# filter dataframe
df = df[~df.variable_name.isin(['Total Housing Units', 'Total Housing Units: Occupied', 'Total Housing Units: Vacant' ])]

# setup new fields
df['Housing Units'] = df['variable_name']
# setup a new Year column and format it
df['Year'] = pd.to_datetime(df['year_sample'], format='%Y') 
df['Year']   = df['Year'].dt.strftime('%Y')
# setup a new Count field based on the summed values
df['Count'] = df['value']
# cast count as integer type
df = df.astype({'Count':int})
df['Percent'] = 0.0
df_test = df
# keep the new columns
df = df[['Housing Units','Year','Percent','Count']]

df90 = df[df['Year']== '1990']
df00 = df[df['Year']== '2000']
df10 = df[df['Year']== '2010']
df20 = df[df['Year']== '2020']

# calculate percents of total for each year
df.loc[df['Year'] == '1990', 'Percent'] =  (df90['Count'] / df90['Count'].sum()) * 100
df.loc[df['Year'] == '2000', 'Percent'] =  (df00['Count'] / df00['Count'].sum()) * 100
df.loc[df['Year'] == '2010', 'Percent'] =  (df10['Count'] / df10['Count'].sum()) * 100
df.loc[df['Year'] == '2020', 'Percent'] =  (df20['Count'] / df20['Count'].sum()) * 100

pivotRace = pd.pivot_table(df,index=['Year'],
                              columns='Housing Units',
                              values='Percent')

flat = pd.DataFrame(pivotRace.to_records())
df = flat

# dcitionary to map old column names to new column names
new_column_names = {
    'Occupied Housing Units: Owner Occupied': 'Owner Occupied',
    'Occupied Housing Units: Renter Occupied': 'Renter Occupied',
    'Vacant Housing Units: Other': 'Vacant: other',
    'Vacant Housing Units: Seasonal, recreational, or occasional use': 'Vacant: seasonal or occasional use'
}


# Rename the columns
df = df.rename(columns=new_column_names)

# get list of columns to stack in bar chart
houseList = df.columns.to_list()
houseList.remove('Year')

In [ ]:
color_mapping = { 'Owner Occupied' : "#996308",
 'Renter Occupied' : "#4D590D",
 'Vacant: other' : "#BAC4E3",
 'Vacant: seasonal or occasional use' : "#80638F"
}
# setup the figure
fig = px.bar(df, x="Year", 
             y=houseList, 
             color_discrete_map=color_mapping
             )

fig.update_traces(hovertemplate='<br>%{y:.2f}%')

# set layout
fig.update_layout(title='Housing Units', 
                    font_family=font,
                    template=template,
                    showlegend=True,
                    legend_title='',
                    hovermode="x unified",
                    xaxis = dict(
                        type='category',
                        tickmode = 'linear'
                    ),
                    yaxis = dict(
                        tickmode = 'linear',
                        tick0 = 0,
                        dtick = 10,
                        range=[0, 100],
                        title_text='Percent'
                    )
                 )

fig.show()
fig.write_html(os.path.join(workspace, "Demographics_Tahoe_HousingUnits.html"))



In [ ]:
grouped_neighborhood = neighborhood_data.groupby("STATE")['Neighborhood_Population_2020'].sum().reset_index()
# Add Formatting

color_mapping = {
    'NV': '#ed954d',
    'CA': '#07467c'
}

fig = px.bar(grouped_neighborhood, x='STATE', 
             y="Neighborhood_Population_2020", color = 'STATE', opacity = .90,
              labels={'Category': 'STATE', 'Value': 'Neighborhood_Population_2020'},
             color_discrete_map=color_mapping

             )
grouped_neighborhood['legend_column'] = grouped_neighborhood.apply(lambda row: f"{row['STATE']} ({int(row['Neighborhood_Population_2020']):,d} Residents)", axis=1)
custom_names = grouped_neighborhood.set_index('STATE')['legend_column'].to_dict()
fig.for_each_trace(lambda t: t.update(name = custom_names[t.name]))

fig.update_layout(
    title='',
    title_x= 0.5,
    xaxis_title='',
    yaxis_title='',
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(
        title='State',  # Legend title
        
        x=0.85,  # X-position of the legend
        y=1,  # Y-position of the legend
        traceorder = 'reversed',
        font=dict(size=16)
    )
    #legend_title='State',
    #legend_traceorder='reversed'
)
fig.update_xaxes(showticklabels=False)
fig.update_layout(showlegend=False)

fig.update_layout(yaxis=dict( tickfont=dict(size=64)))


image_width = 1000
image_height = 1000
fig.show()

#Maybe update this for 

fig.write_html(os.path.join(workspace, "census_presentation_state.html"))
pio.write_image(fig, 'census_presentation_state.png', width=image_width, height=image_height)


In [ ]:
import kaleido
import plotly.io as pio

color_mapping={
'Douglas County': '#674A40',
'El Dorado County': '#50A3A4',
'Placer County': '#FCAF38',
'Washoe County': '#F95335',
}

grouped_neighborhood = neighborhood_data.groupby("County")['Neighborhood_Population_2020'].sum().reset_index()
fig = px.bar(grouped_neighborhood, x='County', 
             y="Neighborhood_Population_2020", color = 'County',
             color_discrete_map=color_mapping, opacity = .90,
              labels={'Category': 'County', 'Value': 'Neighborhood_Population_2020'}
             )

grouped_neighborhood['legend_column'] = grouped_neighborhood.apply(lambda row: f"{row['County']} ({int(row['Neighborhood_Population_2020']):,d} Residents)", axis=1)
custom_names = grouped_neighborhood.set_index("County")['legend_column'].to_dict()
fig.for_each_trace(lambda t: t.update(name = custom_names[t.name]))

fig.update_layout(
    title='',
    title_x= 0.5,
    #title_font = dict(size=40),
    xaxis_title='',
    yaxis_title='',
    plot_bgcolor='rgba(0,0,0,0)'
    
)
fig.update_xaxes(showticklabels=False)
fig.update_layout(showlegend=False)

fig.update_layout(yaxis=dict( tickfont=dict(size=64)))

fig.show()
image_width = 1000
image_height = 1000

fig.write_html(os.path.join(workspace, "census_presentation_county.html"))
pio.write_image(fig, 'bar_chart.png', width=image_width, height=image_height)

In [ ]:


neighborhood_detail = ['Central Incline Village', 'South Lake Tahoe', 'Crystal Bay /surrounding Incline Village', 'Kings Beach /Brockway', 
                       'Glenbrook/ Kingsbury/ E. Shore Douglas', 'Meyers / Hope Valley / Luther Pass']
neighborhood_data['Grouped_Neighborhoods'] = np.where(neighborhood_data['NEIGHBORHOOD'].isin(neighborhood_detail), neighborhood_data['NEIGHBORHOOD'], 'Other')
color_mapping = {'Central Incline Village' : '#E8631E', 'South Lake Tahoe':'#88d8b0', 'Crystal Bay /surrounding Incline Village':'#ffeead', 'Kings Beach /Brockway':'#ff6f69', 
                       'Glenbrook/ Kingsbury/ E. Shore Douglas': '#5FB1E4', 'Meyers / Hope Valley / Luther Pass':'#ffcc5c', 'Other':'#828282'

}

#neighborhood_data['Grouped_Neighborhoods'] = neighborhood_data['NEIGHBORHOOD']
grouped_neighborhood = neighborhood_data.groupby('Grouped_Neighborhoods')['Neighborhood_Population_2020'].sum().reset_index()

neighborhood_order = ['Central Incline Village', 'Crystal Bay /surrounding Incline Village',  'Glenbrook/ Kingsbury/ E. Shore Douglas', 'Kings Beach /Brockway', 
                       'Meyers / Hope Valley / Luther Pass', 'South Lake Tahoe', 'Other']

# Sort the DataFrame based on the custom order of the 'Name' column
grouped_neighborhood = grouped_neighborhood.sort_values(by=['Grouped_Neighborhoods'], key=lambda x: x.map({v: i for i, v in enumerate(neighborhood_order)}))

grouped_neighborhood.reset_index(drop=True, inplace=True)


fig = px.bar(grouped_neighborhood, x='Grouped_Neighborhoods', 
             y="Neighborhood_Population_2020", color = 'Grouped_Neighborhoods',
             color_discrete_map=color_mapping, opacity = .90,
              labels={'Category': 'Grouped_Neighborhoods', 'Value': 'Neighborhood_Population_2020'}
             )

grouped_neighborhood['legend_column'] = grouped_neighborhood.apply(lambda row: f"{row['Grouped_Neighborhoods']} ({int(row['Neighborhood_Population_2020']):,d} Residents)", axis=1)
custom_names = grouped_neighborhood.set_index('Grouped_Neighborhoods')['legend_column'].to_dict()
fig.for_each_trace(lambda t: t.update(name = custom_names[t.name]))

#fig.update_layout(xaxis={'categoryorder':'total ascending'})

fig.update_layout(
    title='',
    title_x= 0.5,
    title_font = dict(size=32),
    xaxis_title='',
    yaxis_title='Population',
    plot_bgcolor='rgba(0,0,0,0)'
    
)
fig.update_xaxes(showticklabels=False)
#fig.update_layout(showlegend=False)

fig.update_layout(
    title='',
    title_x= 0.5,
    #title_font = dict(size=40),
    xaxis_title='',
    yaxis_title='',
    plot_bgcolor='rgba(0,0,0,0)'
    
)
fig.update_xaxes(showticklabels=False)
fig.update_layout(showlegend=False)

fig.update_layout(yaxis=dict( tickfont=dict(size=64)))

fig.show()
image_width = 1000
image_height = 1000
fig.write_html(os.path.join(workspace, "census_presentation_community.html"))
pio.write_image(fig, 'community_chart.png', width=image_width, height=image_height)

#### Tenancy

### Housing Tenancy Chart

#### Values

#### Rents

### Income

## Median Income graphs

### Import median income data

In [ ]:
service_url = 'https://maps.trpa.org/server/rest/services/Demographics/FeatureServer/18'

feature_layer = FeatureLayer(service_url)
query_result = feature_layer.query()
# Convert the query result to a list of dictionaries
feature_list = query_result.features

# Create a pandas DataFrame from the list of dictionaries
all_data = pd.DataFrame([feature.attributes for feature in feature_list])

acs_median_income_data = all_data.loc[(all_data['Category']=='Household Income (2021 Inflation Adjusted Dollars)')&(all_data['Year']>2010)]
counties = ['El Dorado County (Tahoe Basin)','Douglas County (Tahoe Basin)', 'Placer County (Tahoe Basin)', 'Washoe County (Tahoe Basin)']
acs_median_income_data_county = acs_median_income_data.loc[acs_median_income_data['Geography'].isin(counties)]
acs_median_income_data_north_south = acs_median_income_data.loc[acs_median_income_data['Geography'].isin(['North Lake', 'South Lake'])]
acs_median_income_data_Basin = acs_median_income_data.loc[acs_median_income_data['Geography'].isin(['Basin'])]

### Median Income Bar Chart

In [ ]:
fig_bar = px.bar(acs_median_income_data_Basin, x="Year", y="Value",
                labels=dict(Year='Year', Value='Median Income'),
                title="Tahoe Basin Median Household Income",
)
fig_bar.update_layout(font_family=font,
                 template=template,
                    hoverlabel=dict(
                    bgcolor='white',
                       font=dict(color='#6680A8') 
                    ),
    
                     title=dict(
                                x=0.5,
                                xanchor='center',
                                font=dict(size=32)),
                     xaxis=dict(
                                tick0=2012, 
                                dtick=2,
                                title = 'Year',
							                  title_font_size = 24,
                                tickfont = dict(size=20)),
                     yaxis=dict(
                                range=[0,100000],
                                title = 'Household Income <br><sub>(2021 inflation adjusted dollars)',
							                  title_font_size = 24,
                                tickfont = dict(size=20)))
fig_bar.update_traces(marker_color='#2F3F56',
                     hovertemplate='Census Year: %{x} <br> Median Income: %{y:,.0f} ')
fig_bar.show()



fig_bar.write_html(os.path.join(workspace, "Open_Data_Figures\Demographics_Tahoe_Median_Income_Bar.html"))

In [ ]:
fig_line = px.line(acs_median_income_data_Basin, x='Year', y='Value',
                   labels=dict(Year='Year', Value='Median Income'),
                title="Tahoe Median Income (2021 Dollars)")
fig_line.update_layout(font_family=font,
                 template=template,
                    hoverlabel=dict(
                    bgcolor='white',
                       font=dict(color='#6680A8') 
                    ),
                    title=dict(
                    x=0.5,
                xanchor='center'),
                    yaxis=dict(  range=[0,90000])
                    )
fig_line.update_traces(marker_color='#2F3F56',
                     hovertemplate='Census Year: %{x} <br> Median Income: %{y:,.0f} ')
fig_line.show()

fig_line.write_html(os.path.join(workspace, "Demographics_Tahoe_Median_Income_Line.html"))

In [ ]:
counties = ['El Dorado County (Tahoe Basin)','Douglas County (Tahoe Basin)', 'Placer County (Tahoe Basin)', 'Washoe County (Tahoe Basin)', 'Basin']

acs_median_income_data_county = acs_median_income_data.loc[acs_median_income_data['Geography'].isin(counties)]

new_labels = {
    'El Dorado County (Tahoe Basin)': 'El Dorado',
    'Douglas County (Tahoe Basin)': 'Douglas', 
    'Placer County (Tahoe Basin)': 'Placer', 
    'Washoe County (Tahoe Basin)': 'Washoe', 
    'Basin': 'Basin (Median)'
}

color_mapping = {
    'El Dorado':'#7f92cb',
    'Douglas':'#3e450a', 
    'Placer':'#664f72', 
    'Washoe':'#aa6a00', 
    'Basin (Median)':'#2F3F56'
}

acs_median_income_data_county['plot_labels'] = acs_median_income_data_county['Geography'].map(new_labels)

fig_line = px.line(acs_median_income_data_county, x='Year', y='Value', color='plot_labels', color_discrete_map= color_mapping,
                   hover_data=['Geography'],
                   labels=dict(Year='Year', Value='Median Income'),
                title="Median Household Income within Basin by County")
fig_line.update_layout(font_family=font,
                 template=template,
                    hoverlabel=dict(
                    bgcolor='white',
                       font=dict(color='#6680A8') 
                    ),
    
                     title=dict(
                                x=0.5,
                                xanchor='center',
                                font=dict(size=32)),
                     xaxis=dict(
                                tick0=2012, 
                                dtick=2,
                                title = 'Year',
							                  title_font_size = 24,
                                tickfont = dict(size=20)),
                     yaxis=dict(
                                range=[0,140000],
                                title = 'Household Income <br><sub>(2021 inflation adjusted dollars)',
							                  title_font_size = 24,
                                tickfont = dict(size=20)),

    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(
        title='County',  # Legend title
        
        
        font=dict(size=16)
    )
    #legend_title='State',
    #legend_traceorder='reversed'

                    )
fig_line.update_traces(hovertemplate='County: %{customdata[0]} <br> Census Year: %{x} <br> Median Income: $%{y:,.0f}<extra></extra>' )
fig_line.update_traces(line=dict(dash="dash"), selector=dict(name="Basin (Median)"), opacity = 1)


fig_line.show()

fig_line.write_html(os.path.join(workspace, "Open_Data_Figures\Demographics_Tahoe_County_Median_Income_Line.html"))

# Census Presentation Graphs

## Import Data 

In [ ]:
#Get Data
service_url = 'https://maps.trpa.org/server/rest/services/Demographics/FeatureServer/28'

feature_layer = FeatureLayer(service_url)
query_result = feature_layer.query()
# Convert the query result to a list of dictionaries
feature_list = query_result.features

# Create a pandas DataFrame from the list of dictionaries
all_data = pd.DataFrame([feature.attributes for feature in feature_list])

#### Income Distribution By Year

In [ ]:
income_data = all_data.loc[(all_data['sample_level']=='tract')&(all_data['county']!='510')&
                           (all_data['variable_category']=='Household Income')&
                           (all_data['dataset']=='acs/acs5')]
income_data = income_data.loc[income_data['year_sample']>2015]

color_mapping = {
    'Under $40k/Yr': '#8E8971',
    '$40k to $75k': '#b09763',
'$75k to $150k': '#405d74',
'Over $150k': '#935f59'
}

grouped_income_categories = {
    '$10,000 to $14,999' : 'Under $40k/Yr',
'$100,000 to $124,999' : '$75k to $150k',
'$125,000 to $149,999' : '$75k to $150k',
'$15,000 to $19,999' : 'Under $40k/Yr',
'$150,000 to $199,999' : 'Over $150k',
'$20,000 to $24,999' : 'Under $40k/Yr',
'$200,000 or more' : 'Over $150k',
'$25,000 to $29,999' : 'Under $40k/Yr',
'$30,000 to $34,999' : 'Under $40k/Yr',
'$35,000 to $39,999' : 'Under $40k/Yr',
'$40,000 to $44,999' : '$40k to $75k',
'$45,000 to $49,999' : '$40k to $75k',
'$50,000 to $59,999' : '$40k to $75k',
'$60,000 to $74,999' : '$40k to $75k',
'$75,000 to $99,999' : '$75k to $150k',
'Less than $10,000' : 'Under $40k/Yr'

}

income_data['grouped_income'] = income_data['variable_name'].map(grouped_income_categories)
income_data['Year'] = income_data['year_sample'].astype(int).astype('str')


grouped_income_data = income_data.groupby(['Year', 'grouped_income'], as_index=False)['value'].sum()

# Define a custom sorting order for the 'Name' column
custom_order = ['Under $40k/Yr', '$40k to $75k', '$75k to $150k', 'Over $150k']

# Sort the DataFrame based on the custom order of the 'Name' column
grouped_income_data = grouped_income_data.sort_values(by=['grouped_income'], key=lambda x: x.map({v: i for i, v in enumerate(custom_order)}))

grouped_income_data.reset_index(drop=True, inplace=True)

custom_order = ['2016', '2017', '2018', '2019', '2020', '2021']
font = "Arial"
# Sort the DataFrame based on the custom order of the 'Name' column
grouped_income_data = grouped_income_data.sort_values(by=['Year'], key=lambda x: x.map({v: i for i, v in enumerate(custom_order)}))

grouped_income_data.reset_index(drop=True, inplace=True)


fig_bar = px.bar(grouped_income_data, x="Year", y="value", color = 'grouped_income', barmode='group',
                 color_discrete_map=color_mapping,
                #labels=dict(Year='Year', Value='Median Income'),
                title="Tahoe Household Income")
                 #color_discrete_sequence=px.colors.qualitative.Safe)
                #labels=dict(Year='Year', Value='Median Income'),
                #title="Tahoe Median Income (2021 Dollars)",

# Replace Hover Text
fig_bar.update_layout(font_family=font,
                 template=template,
                 bargroupgap = 0.10,
                    hoverlabel=dict(
                    bgcolor='white',
                  
                       font=dict(color='#6680A8') 
                    ),
                title=dict(
                    x=0.5,
                    xanchor='center',
                    font=dict(size=24)
                    ),
    legend=dict(
        title='Household Income',  # Legend title
        font=dict(size=16)
        ), xaxis_title='',
    yaxis_title='',yaxis=dict(
        tickformat=",.0f"
),

)


fig_bar.update_yaxes(
    range=[0, 8000]  # Set the x-axis range
)
#fig_bar.update_traces(marker_color='#2F3F56',
#                     hovertemplate='Census Year: %{x} <br> Median Income: %{y:,.0f} ')
fig_bar.show()

image_width = 1800
image_height = 600

fig_bar.write_html(os.path.join(workspace, "Demographics_Tahoe_Income_Categories_Bar_Year.html"))

#pio.write_image(fig_bar, 'Chart_Outputs\Demographics_Tahoe_Income_Categories_Bar_Year.png', width=image_width, height=image_height)


In [ ]:
custom_order = ['2016', '2017', '2018', '2019', '2020', '2021']
font = "Arial"
# Sort the DataFrame based on the custom order of the 'Name' column
grouped_income_data = grouped_income_data.sort_values(by=['Year'], key=lambda x: x.map({v: i for i, v in enumerate(custom_order)}))

grouped_income_data.reset_index(drop=True, inplace=True)
color_mapping = {
    '2016':'#7e8d98',
    '2017': '#8E8971',
    '2018': '#b09763',
'2019': '#405d74',
'2020': '#935f59',
'2021':'#E6D28C'
}

fig_bar = px.bar(grouped_income_data, x='grouped_income', y="value", color = 'Year', barmode='group',
                 color_discrete_map=color_mapping,
                #labels=dict(Year='Year', Value='Median Income'),
                title="Tahoe Region Households by Household Income")
                 #color_discrete_sequence=px.colors.qualitative.Safe)
                #labels=dict(Year='Year', Value='Median Income'),
                #title="Tahoe Median Income (2021 Dollars)",

# Replace Hover Text
fig_bar.update_layout(font_family=font,
                 template=template,
                 bargroupgap = 0.10,
                    hoverlabel=dict(
                    bgcolor='white',
                  
                       font=dict(color='#6680A8') 
                    ),
                title=dict(
                    x=0.5,
                    font= dict(size=24),
                    xanchor='center'
                    ),
    legend=dict(
        title='Year',  # Legend title
        font=dict(size=16)
        ), xaxis_title='',

    yaxis_title='',yaxis=dict(
        tickformat=",.0f"
),

)

fig_bar.update_xaxes(
    title_font=dict(size=16), tickfont = dict(size=20)
)
fig_bar.update_yaxes(
    range=[0, 8000]  # Set the x-axis range
)
#fig_bar.update_traces(marker_color='#2F3F56',
#                     hovertemplate='Census Year: %{x} <br> Median Income: %{y:,.0f} ')
fig_bar.show()



fig_bar.write_html(os.path.join(workspace, "Demographics_Tahoe_Income_Categories_Bar.html"))

### Age distribution

In [ ]:
age_data = all_data.loc[(all_data['sample_level']=='tract')&(all_data['county']!='510')&
                           (all_data['variable_category']=='TRPA 10-year Age Categories Grouped: Age')&
                           (all_data['variable_name']!='Total')]
#age_data = age_data.loc[age_data['year_sample']>2015]


Age_Mapping = {'10 to 19' : 'Under 30',
'20 to 29' : 'Under 30',
'30 to 39' : 'Ages 30-49',
'40 to 49' : 'Ages 30-49',
'50 to 59' : 'Age 50 and Over',
'60 to 69' : 'Age 50 and Over',
'70 to 79' : 'Age 50 and Over',
'Over 80' : 'Age 50 and Over',
'Under 10' : 'Under 30'}

color_mapping = {
    'Under 30': '#b09763',
'Ages 30-49': '#405d74',
'Age 50 and Over': '#935f59'
}

age_data['grouped_age'] = age_data['variable_name'].map(Age_Mapping)
age_data['Year'] = age_data['year_sample'].astype(int).astype('str')

grouped_age_data = age_data.groupby(['Year', 'grouped_age'], as_index=False)['value'].sum()

# Define a custom sorting order for the 'Name' column
custom_order = ['Under 30', 'Ages 30-49', 'Age 50 and Over']

# Sort the DataFrame based on the custom order of the 'Name' column
grouped_age_data = grouped_age_data.sort_values(by=['grouped_age'], key=lambda x: x.map({v: i for i, v in enumerate(custom_order)}))

grouped_age_data.reset_index(drop=True, inplace=True)

fig_bar = px.bar(grouped_age_data, x="Year", y="value", color = 'grouped_age', barmode='group',
                 color_discrete_map=color_mapping, opacity = .9,
                #labels=dict(Year='Year', Value='Median Income'),
                title="Tahoe is Getting Older")

fig_bar.update_layout(font_family=font,
                 template=template,
                    hoverlabel=dict(
                    bgcolor='white',
                       font=dict(color='#6680A8') 
                    ),
                    title=dict(
                    x=0.5,
                xanchor='center', font=dict(size=36)), bargroupgap = 0.10,
    legend=dict(
        title='',  # Legend title
        font=dict(size=18),
        orientation = 'h',
        yanchor = 'bottom',
        y=-.4,
        x=.3), xaxis_title='',
    yaxis_title='Number of Residents',yaxis=dict(
        tickformat=",.0f"  # Format ticks as full numbers without decimal places and commas for thousands separator
    ))

fig_bar.update_xaxes(
    title_font=dict(size=16), tickfont = dict(size=20)
)
fig_bar.update_yaxes(
    title_font=dict(size=16), tickfont = dict(size=20)
)
#fig_bar.update_traces(marker_color='#2F3F56',
#                     hovertemplate='Census Year: %{x} <br> Median Income: %{y:,.0f} ')
fig_bar.show()



fig_bar.write_html(os.path.join(workspace, "Demographics_Tahoe_age_Categories_Bar_Year.html"))


In [ ]:
age_data = all_data.loc[(all_data['sample_level']=='tract')&(all_data['county']!='510')&
                           (all_data['variable_category']=='TRPA 10-year Age Categories Grouped: Age')&
                           (all_data['variable_name']!='Total')]
#age_data = age_data.loc[age_data['year_sample']>2015]


Age_Mapping = {'10 to 19' : 'Under 30',
'20 to 29' : 'Under 30',
'30 to 39' : 'Ages 30-49',
'40 to 49' : 'Ages 30-49',
'50 to 59' : 'Age 50 and Over',
'60 to 69' : 'Age 50 and Over',
'70 to 79' : 'Age 50 and Over',
'Over 80' : 'Age 50 and Over',
'Under 10' : 'Under 30'}
color_mapping = {
    '1990': '#8E8971',
    '2000': '#b09763',
'2010': '#405d74',
'2020': '#935f59'
}
age_data['grouped_age'] = age_data['variable_name'].map(Age_Mapping)
age_data['Year'] = age_data['year_sample'].astype(int).astype('str')

grouped_age_data = age_data.groupby(['Year', 'grouped_age'], as_index=False)['value'].sum()

# Define a custom sorting order for the 'Name' column
custom_order = ['Under 30', 'Ages 30-49', 'Age 50 and Over']

# Sort the DataFrame based on the custom order of the 'Name' column
grouped_age_data = grouped_age_data.sort_values(by=['grouped_age'], key=lambda x: x.map({v: i for i, v in enumerate(custom_order)}))

grouped_age_data.reset_index(drop=True, inplace=True)

fig_bar = px.bar(grouped_age_data, x="grouped_age", y="value", color = 'Year', barmode='group',
 color_discrete_map=color_mapping, opacity = .9,
                #labels=dict(Year='Year', Value='Median Income'),
                title="Tahoe is Getting Older")

fig_bar.update_layout(font_family=font,
                 template=template,
                    hoverlabel=dict(
                    bgcolor='white',
                       font=dict(color='#6680A8') 
                    ),
                    title=dict(
                    x=0.5,
                xanchor='center', font=dict(size=36)), bargroupgap = 0.10,
    legend=dict(
        title='',  # Legend title
        font=dict(size=18),
        orientation = 'h',
        yanchor = 'bottom',
        y=-.4,
        x=.3), xaxis_title='',
    yaxis_title='',yaxis=dict(
        tickformat=",.0f"  # Format ticks as full numbers without decimal places and commas for thousands separator
    ))

fig_bar.update_xaxes(
    title_font=dict(size=16), tickfont = dict(size=20)
)
fig_bar.update_yaxes(
    title_font=dict(size=16), tickfont = dict(size=20)
)
#fig_bar.update_traces(marker_color='#2F3F56',
#                     hovertemplate='Census Year: %{x} <br> Median Income: %{y:,.0f} ')
fig_bar.show()




fig_bar.write_html(os.path.join(workspace, "Demographics_Tahoe_age_Categories_Bar_category.html"))


#### Source of Income

#### Housing Affordability

### Workforce

#### Place of Work

#### Travel Time to Work

### Jobs

#### Unemployment

#### Employment

In [ ]:
#Employment numbers
dfEmployment = pd.read_sql("SELECT * FROM sde_tabular.SDE.Demographics where category = 'Employment'", conn)
dfEmployment = dfEmployment.sort_values(by=['Year'], ignore_index=True)

fig_Employment = px.line(dfEmployment, x="Year", y="Value",
                labels=dict(Year='Year', Value='Employment'),
                title="Tahoe Basin Demographics")

fig_Employment.update_traces(mode='markers+lines', 
                     patch={"line": {
                           "shape": "spline"}},
                     hovertemplate='Census Year: %{x} <br> Population: %{y:,.0f} '
                    )
fig_Employment.update_layout(font_family=font,
                 template=template,
                    hoverlabel=dict(
                    bgcolor='white',
                       font=dict(color='blue') 
                    ),
                    title=dict(
                    x=0.5,
                xanchor='center'),
                    xaxis=dict(tick0=2000, dtick=10)
                            )

fig_Employment.show()

### Industries

#### Establishements